## using resnet18 to process image data classification

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import warnings
warnings.filterwarnings("ignore")

In [17]:
# train = pd.read_csv("/kaggle/input/uw-cs480-fall20/train.csv")
# test = pd.read_csv("/kaggle/input/uw-cs480-fall20/test.csv")

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [18]:
train["jpg"] = "F:/20Fall-master/ML-cs680/kaggle/uw-cs480-fall20-new/images/shuffled-images/" + train['id'].astype('str') + ".jpg"
category_list = train["category"].unique()
category_list.sort()
category_mapping = {label:idx for idx,label in enumerate(category_list)}
train['category_num'] = train['category'].map(category_mapping)

train_dataset = train[{'jpg','category_num'}]
train_dataset = pd.DataFrame(train_dataset, columns = ['jpg','category_num'])
train_dataset.to_csv("train.txt",index = None, header=None)
train_dataset[20000:].to_csv("val.txt",index = None, header=None)

In [78]:
category_mapping

{'Accessories': 0,
 'Apparel Set': 1,
 'Bags': 2,
 'Belts': 3,
 'Bottomwear': 4,
 'Cufflinks': 5,
 'Dress': 6,
 'Eyewear': 7,
 'Flip Flops': 8,
 'Fragrance': 9,
 'Free Gifts': 10,
 'Headwear': 11,
 'Innerwear': 12,
 'Jewellery': 13,
 'Lips': 14,
 'Loungewear and Nightwear': 15,
 'Makeup': 16,
 'Nails': 17,
 'Sandal': 18,
 'Saree': 19,
 'Scarves': 20,
 'Shoes': 21,
 'Socks': 22,
 'Ties': 23,
 'Topwear': 24,
 'Wallets': 25,
 'Watches': 26}

In [79]:
index_mapping = {idx:label for idx,label in enumerate(category_list)}
index_mapping

{0: 'Accessories',
 1: 'Apparel Set',
 2: 'Bags',
 3: 'Belts',
 4: 'Bottomwear',
 5: 'Cufflinks',
 6: 'Dress',
 7: 'Eyewear',
 8: 'Flip Flops',
 9: 'Fragrance',
 10: 'Free Gifts',
 11: 'Headwear',
 12: 'Innerwear',
 13: 'Jewellery',
 14: 'Lips',
 15: 'Loungewear and Nightwear',
 16: 'Makeup',
 17: 'Nails',
 18: 'Sandal',
 19: 'Saree',
 20: 'Scarves',
 21: 'Shoes',
 22: 'Socks',
 23: 'Ties',
 24: 'Topwear',
 25: 'Wallets',
 26: 'Watches'}

In [19]:
val_size = int(len(train)/5)

train1 = train_dataset[val_size:]
train1.to_csv("train1.txt",index = None, header=None)
val1 = train_dataset[:val_size]
val1.to_csv("val1.txt",index = None, header=None)

train2 = pd.concat([train_dataset.iloc[:val_size,], train_dataset.iloc[val_size*2:]])
train2.to_csv("train2.txt",index = None, header=None)
val2 = train_dataset[val_size:val_size*2]
val2.to_csv("val2.txt",index = None, header=None)

train3 = pd.concat([train_dataset.iloc[:val_size*2,], train_dataset.iloc[val_size*3:]])
train3.to_csv("train3.txt",index = None, header=None)
val3 = train_dataset[val_size*2:val_size*3]
val3.to_csv("val3.txt",index = None, header=None)

train4 = pd.concat([train_dataset.iloc[:val_size*3,], train_dataset.iloc[val_size*4:]])
train4.to_csv("train4.txt",index = None, header=None)
val4 = train_dataset[val_size*3:val_size*4]
val4.to_csv("val4.txt",index = None, header=None)

train5 = train_dataset.iloc[:val_size*4,]
train5.to_csv("train5.txt",index = None, header=None)
val5 = train_dataset[val_size*4:]
val5.to_csv("val5.txt",index = None, header=None)


In [20]:
from PIL import Image
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, txt_path, transform = None, target_transform = None):
        fh = open(txt_path, 'r')
        imgs = []
        for line in fh:
            line = line.rstrip()
            words = line.split(",")
#             print(words)
            imgs.append((words[0], int(words[1])))
            self.imgs = imgs 
            self.transform = transform
            self.target_transform = target_transform
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(fn).convert('RGB') 
        if self.transform is not None:
            img = self.transform(img) 
        return img, label
    def __len__(self):
        return len(self.imgs)

In [44]:
BATCH_SIZE = 256

# 准备数据集并预处理
transform_train = transforms.Compose([
    transforms.RandomCrop(48, padding=4),  #先四周填充0，再把图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), #R,G,B每层的归一化用到的均值和方差
])

transform_test = transforms.Compose([
    transforms.CenterCrop(48),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = MyDataset("train.txt", transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

valset = MyDataset("val.txt", transform = transform_test)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

In [71]:
test["jpg"] =  "F:/20Fall-master/ML-cs680/kaggle/uw-cs480-fall20-new/images/shuffled-images/" + test['id'].astype('str') + ".jpg"

test_dataset = test[{'jpg'}]
test_dataset.to_csv("test.txt",index = None, header=None)

class testDataset(Dataset):
    def __init__(self, txt_path, transform = None, target_transform = None):
        fh = open(txt_path, 'r')
        imgs = []
        for line in fh:
            line = line.rstrip()
            words = line.split()
#             print(words)
            imgs.append((words[0]))
            self.imgs = imgs 
            self.transform = transform
            self.target_transform = target_transform
    def __getitem__(self, index):
        fn= self.imgs[index]
        img = Image.open(fn).convert('RGB') 
        if self.transform is not None:
            img = self.transform(img) 
        return img
    def __len__(self):
        return len(self.imgs)
    

testset = testDataset("test.txt", transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=0)

In [63]:
for data in testloader:
    break

In [66]:
outputs = net_total(data)
_, predicted = torch.max(outputs.data, 1)

In [67]:
predicted

test_dataset = test[{'jpg'}]
test_ = "F:/20Fall-master/ML-cs680/kaggle/uw-cs480-fall20-new/images/shuffled-images/1163.jpg"
test_dataset.to_csv("test.txt",index = None, header=None)

tensor([12, 24,  3, 21, 12, 21, 24,  7, 26, 21, 12, 24, 23, 12, 22, 25, 26, 26,
         4, 21, 17, 24,  4,  3,  3, 24,  2, 21, 25, 24, 24, 25, 24,  4, 24,  7,
        26, 25, 26, 21, 24, 24,  4, 24, 26, 24,  8, 24, 24,  2, 26, 19, 12, 24,
        24, 21, 25,  4, 12, 26, 21, 21, 21, 21,  6, 21, 24,  4, 24, 26, 22, 26,
        24,  8, 26, 24, 24, 12, 24, 21, 24, 24, 24, 24, 26, 25,  4, 26,  7, 21,
        24, 24, 21,  8,  9, 24, 21, 12, 21, 24, 21, 26,  5,  9,  4, 24, 21,  2,
        18, 24, 13, 21, 24, 19,  2, 24, 21,  9, 24,  2, 24, 26,  2, 24, 12, 24,
        24,  9, 11,  4, 24, 24, 24, 24, 24, 21, 25,  8, 24, 24, 24, 18, 13, 24,
        24, 24, 17, 24,  2, 24, 12,  2, 24,  6, 24, 21,  7,  4, 24, 24, 26,  9,
        17,  2, 24, 24, 18, 14, 24,  6, 24, 24, 24,  2, 26, 24, 24, 24, 24, 24,
         8, 24, 24, 19, 26, 13, 24, 14, 24,  8, 22, 25, 21, 14, 24, 24, 24, 17,
        24,  4, 24, 24,  8,  2,  9,  2, 24, 24,  2, 13, 26, 26, 21, 12, 21, 21,
        24, 24, 24, 24,  8, 24,  2, 24, 

In [68]:
testimage = Image.open("F:/20Fall-master/ML-cs680/kaggle/uw-cs480-fall20-new/images/shuffled-images/1163.jpg" ).convert('RGB') 
testimage 

testset prediction

total_test_predict = torch.empty(0).to(device)
    
with torch.no_grad():
    for data in testloader:
        images = data 
        images = images.to(device)
        break
        outputs = net_total(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test_predict = torch.cat([total_test_predict, outputs])
test_pred = pd.DataFrame(total_test_predict.tolist())
# val.to_csv('./val_pred_%d.csv' % (k_fold))

KeyboardInterrupt: 

In [76]:
testimagetensor = transform_test(testimage).to(device)
testimagetensor.reshape([1, 3, 48, 48])

tensor([[[[0.9765, 0.8588, 0.9137,  ..., 0.8510, 0.8824, 0.9922],
          [0.8745, 0.8980, 0.8039,  ..., 0.8118, 0.8275, 0.9529],
          [0.8824, 0.9529, 0.9059,  ..., 0.8667, 0.8667, 0.9922],
          ...,
          [0.9608, 1.0000, 0.9059,  ..., 0.9137, 0.9451, 0.9137],
          [0.8745, 0.8510, 0.8275,  ..., 0.8902, 0.8745, 0.9294],
          [0.8902, 0.9451, 0.9294,  ..., 0.9294, 0.7961, 0.8275]],

         [[1.0000, 0.8902, 0.9451,  ..., 0.8824, 0.8902, 1.0000],
          [0.9059, 0.9294, 0.8353,  ..., 0.8431, 0.8353, 0.9608],
          [0.8980, 0.9608, 0.9373,  ..., 0.8980, 0.8745, 0.9922],
          ...,
          [0.9294, 0.9843, 0.8902,  ..., 0.8980, 0.9294, 0.8980],
          [0.8431, 0.8196, 0.8118,  ..., 0.8510, 0.8353, 0.8902],
          [0.8588, 0.9373, 0.9137,  ..., 0.8902, 0.7569, 0.8118]],

         [[1.0000, 0.8980, 0.9529,  ..., 0.8902, 0.9059, 1.0000],
          [0.8980, 0.9373, 0.8588,  ..., 0.8510, 0.8667, 0.9922],
          [0.8902, 0.9765, 0.9451,  ..., 0

In [77]:
# net_total = ResNet18().eval().to(device)
# net_total.load_state_dict(torch.load('model/net_total.pth', map_location=torch.device('cpu')))

net_total(testimagetensor.reshape([1, 3, 48, 48])) 

tensor([[ 1.5086, -0.5231,  2.9911, -0.7517,  1.4482, -1.7936, -0.1346, -1.2470,
         -2.6284,  0.5849,  1.8474,  0.9578,  0.9574,  0.6092, -1.9532, -0.9067,
          0.0995, -2.9456, -0.8371, -1.2209,  0.1501,  1.3352, -0.5303, -2.9779,
          4.5718, -1.7226,  3.3835]], grad_fn=<AddmmBackward>)

In [53]:
# for the valuation, save prediction

saved_net = ResNet18().to(device)
saved_net.load_state_dict(torch.load('model/net_total.pth',map_location=torch.device('cpu')))

valset = MyDataset("val.txt", transform = transform_test)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2) 

total_val_predict = torch.empty(0).to(device)

with torch.no_grad():
    correct = 0
    total = 0
    for data in valloader:
#         net.eval()
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = saved_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
#         total_val_predict += predicted.tolist()
        total_val_predict = torch.cat([total_val_predict, outputs])
    print('val acc：%.03f' % (int(correct) / int(total)))
val = pd.DataFrame(total_val_predict.tolist())
val.to_csv('./val_pred_%d.csv' % (k_fold))

BrokenPipeError: [Errno 32] Broken pipe

In [42]:
class Block(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(Block, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, Block, num_classes=27):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(Block, 64,  2, stride=1)
        self.layer2 = self.make_layer(Block, 128, 2, stride=2)
        self.layer3 = self.make_layer(Block, 256, 2, stride=2)
        self.layer4 = self.make_layer(Block, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(Block)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 模型定义-ResNet
net = ResNet18().to(device)
# net = ResNet18()
# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
# optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

pre_epoch_list = [0,70,100]# 定义已经遍历数据集的次数
EPOCH_list = [70,100,120]   #遍历数据集次数
LR_list = [0.1,0.01,0.001]        #学习率

def train_resnet(k_fold):
    net = ResNet18().to(device)
    
    trainset = MyDataset("train" + str(k_fold) + ".txt", transform = transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

    valset = MyDataset("val" + str(k_fold) + ".txt", transform = transform_test)
    valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

    best_acc = 0  #2 初始化best test accuracy
    print("Start Training, CNN!")  


    with open("acc.txt", "w") as f:
        with open("log.txt", "w")as f2:
            for ep_num in range(3):
                pre_epoch = pre_epoch_list[ep_num]
                EPOCH = EPOCH_list[ep_num]
                LR = LR_list[ep_num]
                optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

                for epoch in range(pre_epoch, EPOCH):
                    print('\nEpoch: %d' % (epoch + 1))
                    net.train()
                    sum_loss = 0.0
                    correct = 0.0
                    total = 0.0
                    for i, data in enumerate(trainloader, 0):
                        length = len(trainloader)
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        optimizer.zero_grad()

                        # forward + backward
                        outputs = net(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                        # 每训练1个batch打印一次loss和准确率
                        sum_loss += loss.item()
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += predicted.eq(labels.data).cpu().sum()
    #                     print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.03f '
    #                           % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), float(correct) / float(total)))
                        f2.write('%03d  %05d |Loss: %.03f | Acc: %.03f '
                              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1),  float(correct / total)))
                        f2.write('\n')
                        f2.flush()

                    # val
                    print("Waiting Test!")
                    with torch.no_grad():
                        correct = 0
                        total = 0
                        for data in valloader:
                            net.eval()
                            images, labels = data
                            images, labels = images.to(device), labels.to(device)
                            outputs = net(images)
                            _, predicted = torch.max(outputs.data, 1)
                            total += labels.size(0)
                            correct += (predicted == labels).sum()
                        print('val acc：%.03f' % (int(correct) / int(total)))
                        acc = int(correct) / int(total)


                        if acc > best_acc:
                            torch.save(net.state_dict(), './net_%d_%03d.pth' % (k_fold, epoch + 1))
                            best_acc = acc
                            print('Saving model......')

                        f.write("EPOCH=%03d,Accuracy= %.03f" % (epoch + 1, acc))
                        f.write('\n')
                        f.flush()

            print("Training Finished, Total EPOCH=%d" % EPOCH)
    return net

In [17]:
net1 = train_resnet(1)

Start Training, CNN!

Epoch: 1
Waiting Test!
val acc：0.464
Saving model......

Epoch: 2
Waiting Test!
val acc：0.576
Saving model......

Epoch: 3
Waiting Test!
val acc：0.598
Saving model......

Epoch: 4
Waiting Test!
val acc：0.409

Epoch: 5
Waiting Test!
val acc：0.698
Saving model......

Epoch: 6
Waiting Test!
val acc：0.785
Saving model......

Epoch: 7
Waiting Test!
val acc：0.463

Epoch: 8
Waiting Test!
val acc：0.487

Epoch: 9
Waiting Test!
val acc：0.757

Epoch: 10
Waiting Test!
val acc：0.772

Epoch: 11


KeyboardInterrupt: 

In [23]:
net1 = train_resnet(1)

Start Training, CNN!

Epoch: 1


BrokenPipeError: [Errno 32] Broken pipe

In [12]:
"""
Code from https://jsideas.net/snapshot_ensemble/
"""

import warnings

warnings.filterwarnings("ignore")

import math
import torch
from torch.optim.lr_scheduler import _LRScheduler

torch.manual_seed(42)


class CosineAnnealingLR_with_Restart(_LRScheduler):

    def __init__(self, optimizer, T_max, T_mult, model, out_dir, take_snapshot, eta_min=0, last_epoch=-1):
        self.T_max = T_max
        self.T_mult = T_mult
        self.Te = self.T_max
        self.eta_min = eta_min
        self.current_epoch = last_epoch

        self.model = model
        self.out_dir = out_dir
        self.take_snapshot = take_snapshot

        self.lr_history = []

        super(CosineAnnealingLR_with_Restart, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        new_lrs = [self.eta_min + (base_lr - self.eta_min) * (1 + math.cos(math.pi * self.current_epoch / self.Te)) / 2 for base_lr in self.base_lrs]

        self.lr_history.append(new_lrs)
        return new_lrs

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
        self.last_epoch = epoch
        self.current_epoch += 1

        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group["lr"] = lr

        ## restart
        if self.current_epoch == self.Te:
            print("restart at epoch {:03d}".format(self.last_epoch + 1))

            if self.take_snapshot:
                torch.save(
                    {"epoch": self.T_max, "state_dict": self.model.state_dict()}, self.out_dir + "/" + "snapshot_e_{:03d}.pth.tar".format(self.T_max)
                )

            ## reset epochs since the last reset
            self.current_epoch = 0

            ## reset the next goal
            self.Te = int(self.Te * self.T_mult)
            self.T_max = self.T_max + self.Te


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 模型定义-ResNet
net = ResNet18().to(device)
# net = ResNet18()
# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
# optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

pre_epoch_list = [0,40,60]# 定义已经遍历数据集的次数
EPOCH_list = [40,60,70]   #遍历数据集次数
LR_list = [0.1,0.01,0.001]        #学习率

def train_sch(k_fold):
    
    net = ResNet18().to(device)
    trainset = MyDataset("train" + str(k_fold) + ".txt", transform = transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

    valset = MyDataset("val" + str(k_fold) + ".txt", transform = transform_test)
    valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

    best_acc = 0  #2 初始化best test accuracy
    print("Start Training, resnet!")  

    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-3)
    scheduler = CosineAnnealingLR_with_Restart(optimizer, T_max=4, T_mult=1.5, model=net, out_dir="./", take_snapshot=True, eta_min=1e-6)
    
    with open("acc.txt", "w") as f:
        with open("log.txt", "w")as f2:
            for ep_num in range(3):
                pre_epoch = pre_epoch_list[ep_num]
                EPOCH = EPOCH_list[ep_num]
                LR = LR_list[ep_num]
                optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
                
                for epoch in range(pre_epoch, EPOCH):
                    scheduler.step()
                    print('\nEpoch: %d' % (epoch + 1))
                    net.train()
                    sum_loss = 0.0
                    correct = 0.0
                    total = 0.0
                    for i, data in enumerate(trainloader, 0):
                        length = len(trainloader)
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        optimizer.zero_grad()

                        # forward + backward
                        outputs = net(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                        # 每训练1个batch打印一次loss和准确率
                        sum_loss += loss.item()
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += predicted.eq(labels.data).cpu().sum()
                        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.03f '
                              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), float(correct) / float(total)))
                        f2.write('%03d  %05d |Loss: %.03f | Acc: %.03f '
                              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1),  float(correct / total)))
                        f2.write('\n')
                        f2.flush()

                    # val
                    print("Waiting Test!")
                    with torch.no_grad():
                        correct = 0
                        total = 0
                        for data in valloader:
                            net.eval()
                            images, labels = data
                            images, labels = images.to(device), labels.to(device)
                            outputs = net(images)
                            _, predicted = torch.max(outputs.data, 1)
                            total += labels.size(0)
                            correct += (predicted == labels).sum()
                        print('val acc：%.03f' % (int(correct) / int(total)))
                        acc = int(correct) / int(total)


                        if acc > best_acc:
                            torch.save(net.state_dict(), './net_%d.pth' % (k_fold))
                            best_acc = acc
                            print('Saving model......')

                        f.write("EPOCH=%03d,Accuracy= %.03f" % (epoch + 1, acc))
                        f.write('\n')
                        f.flush()

            print("Training Finished, Total EPOCH=%d" % EPOCH)
    return net

In [ ]:
# for the valuation, save prediction
def pred_val(k_fold):
    saved_net = ResNet18().to(device)
    saved_net.load_state_dict(torch.load('./net_%d.pth' % (k_fold)))

    valset = MyDataset("val" + str(k_fold) + ".txt", transform = transform_test)
    valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2) 
        
    total_val_predict = torch.empty(0).to(device)
        
    with torch.no_grad():
        correct = 0
        total = 0
        for data in valloader:
    #         net.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = saved_net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
    #         total_val_predict += predicted.tolist()
            total_val_predict = torch.cat([total_val_predict, outputs])
        print('val acc：%.03f' % (int(correct) / int(total)))
    val = pd.DataFrame(total_val_predict.tolist())
    val.to_csv('./val_pred_%d.csv' % (k_fold))
    return val 

In [29]:
net1_new = train_sch(1)
val1_pred = pred_val(1)

Start Training, resnet!

Epoch: 1
[epoch:1, iter:1] Loss: 3.523 | Acc: 0.105 
[epoch:1, iter:2] Loss: 6.580 | Acc: 0.229 
[epoch:1, iter:3] Loss: 6.310 | Acc: 0.203 
[epoch:1, iter:4] Loss: 5.498 | Acc: 0.169 
[epoch:1, iter:5] Loss: 5.011 | Acc: 0.146 
[epoch:1, iter:6] Loss: 4.631 | Acc: 0.182 
[epoch:1, iter:7] Loss: 4.354 | Acc: 0.210 
[epoch:1, iter:8] Loss: 4.122 | Acc: 0.231 
[epoch:1, iter:9] Loss: 3.944 | Acc: 0.245 
[epoch:1, iter:10] Loss: 3.791 | Acc: 0.251 
[epoch:1, iter:11] Loss: 3.657 | Acc: 0.251 
[epoch:1, iter:12] Loss: 3.557 | Acc: 0.253 
[epoch:1, iter:13] Loss: 3.470 | Acc: 0.260 
[epoch:1, iter:14] Loss: 3.387 | Acc: 0.267 
[epoch:1, iter:15] Loss: 3.327 | Acc: 0.270 
[epoch:1, iter:16] Loss: 3.263 | Acc: 0.278 
[epoch:1, iter:17] Loss: 3.215 | Acc: 0.279 
[epoch:1, iter:18] Loss: 3.169 | Acc: 0.281 
[epoch:1, iter:19] Loss: 3.129 | Acc: 0.286 
[epoch:1, iter:20] Loss: 3.088 | Acc: 0.291 
[epoch:1, iter:21] Loss: 3.057 | Acc: 0.297 
[epoch:1, iter:22] Loss: 3.023

[epoch:3, iter:179] Loss: 1.566 | Acc: 0.562 
[epoch:3, iter:180] Loss: 1.562 | Acc: 0.563 
[epoch:3, iter:181] Loss: 1.561 | Acc: 0.562 
[epoch:3, iter:182] Loss: 1.560 | Acc: 0.563 
[epoch:3, iter:183] Loss: 1.557 | Acc: 0.563 
[epoch:3, iter:184] Loss: 1.555 | Acc: 0.564 
[epoch:3, iter:185] Loss: 1.552 | Acc: 0.564 
[epoch:3, iter:186] Loss: 1.553 | Acc: 0.563 
[epoch:3, iter:187] Loss: 1.548 | Acc: 0.565 
[epoch:3, iter:188] Loss: 1.546 | Acc: 0.565 
[epoch:3, iter:189] Loss: 1.542 | Acc: 0.566 
[epoch:3, iter:190] Loss: 1.540 | Acc: 0.566 
[epoch:3, iter:191] Loss: 1.535 | Acc: 0.568 
[epoch:3, iter:192] Loss: 1.533 | Acc: 0.569 
[epoch:3, iter:193] Loss: 1.530 | Acc: 0.569 
[epoch:3, iter:194] Loss: 1.529 | Acc: 0.570 
[epoch:3, iter:195] Loss: 1.530 | Acc: 0.569 
[epoch:3, iter:196] Loss: 1.526 | Acc: 0.570 
[epoch:3, iter:197] Loss: 1.525 | Acc: 0.571 
[epoch:3, iter:198] Loss: 1.523 | Acc: 0.572 
[epoch:3, iter:199] Loss: 1.522 | Acc: 0.572 
[epoch:3, iter:200] Loss: 1.521 | 

[epoch:6, iter:353] Loss: 0.983 | Acc: 0.720 
[epoch:6, iter:354] Loss: 0.983 | Acc: 0.722 
[epoch:6, iter:355] Loss: 0.984 | Acc: 0.721 
[epoch:6, iter:356] Loss: 0.975 | Acc: 0.724 
[epoch:6, iter:357] Loss: 0.970 | Acc: 0.725 
[epoch:6, iter:358] Loss: 0.972 | Acc: 0.725 
[epoch:6, iter:359] Loss: 0.964 | Acc: 0.728 
[epoch:6, iter:360] Loss: 0.960 | Acc: 0.730 
[epoch:6, iter:361] Loss: 0.964 | Acc: 0.729 
[epoch:6, iter:362] Loss: 0.964 | Acc: 0.728 
[epoch:6, iter:363] Loss: 0.969 | Acc: 0.727 
[epoch:6, iter:364] Loss: 0.966 | Acc: 0.727 
[epoch:6, iter:365] Loss: 0.962 | Acc: 0.729 
[epoch:6, iter:366] Loss: 0.961 | Acc: 0.728 
[epoch:6, iter:367] Loss: 0.959 | Acc: 0.728 
[epoch:6, iter:368] Loss: 0.958 | Acc: 0.728 
[epoch:6, iter:369] Loss: 0.955 | Acc: 0.729 
[epoch:6, iter:370] Loss: 0.954 | Acc: 0.729 
[epoch:6, iter:371] Loss: 0.953 | Acc: 0.729 
[epoch:6, iter:372] Loss: 0.953 | Acc: 0.729 
[epoch:6, iter:373] Loss: 0.950 | Acc: 0.731 
[epoch:6, iter:374] Loss: 0.944 | 

[epoch:8, iter:530] Loss: 0.781 | Acc: 0.774 
[epoch:8, iter:531] Loss: 0.781 | Acc: 0.774 
[epoch:8, iter:532] Loss: 0.781 | Acc: 0.774 
[epoch:8, iter:533] Loss: 0.781 | Acc: 0.774 
[epoch:8, iter:534] Loss: 0.781 | Acc: 0.773 
[epoch:8, iter:535] Loss: 0.781 | Acc: 0.773 
[epoch:8, iter:536] Loss: 0.780 | Acc: 0.773 
[epoch:8, iter:537] Loss: 0.780 | Acc: 0.773 
[epoch:8, iter:538] Loss: 0.780 | Acc: 0.773 
[epoch:8, iter:539] Loss: 0.778 | Acc: 0.774 
[epoch:8, iter:540] Loss: 0.777 | Acc: 0.773 
[epoch:8, iter:541] Loss: 0.775 | Acc: 0.774 
[epoch:8, iter:542] Loss: 0.774 | Acc: 0.774 
[epoch:8, iter:543] Loss: 0.773 | Acc: 0.774 
[epoch:8, iter:544] Loss: 0.773 | Acc: 0.774 
Waiting Test!
val acc：0.755
Saving model......

Epoch: 9
[epoch:9, iter:545] Loss: 0.683 | Acc: 0.777 
[epoch:9, iter:546] Loss: 0.664 | Acc: 0.797 
[epoch:9, iter:547] Loss: 0.683 | Acc: 0.790 
[epoch:9, iter:548] Loss: 0.710 | Acc: 0.781 
[epoch:9, iter:549] Loss: 0.696 | Acc: 0.790 
[epoch:9, iter:550] Los

[epoch:11, iter:703] Loss: 0.585 | Acc: 0.825 
[epoch:11, iter:704] Loss: 0.585 | Acc: 0.825 
[epoch:11, iter:705] Loss: 0.588 | Acc: 0.823 
[epoch:11, iter:706] Loss: 0.583 | Acc: 0.825 
[epoch:11, iter:707] Loss: 0.583 | Acc: 0.824 
[epoch:11, iter:708] Loss: 0.585 | Acc: 0.824 
[epoch:11, iter:709] Loss: 0.587 | Acc: 0.824 
[epoch:11, iter:710] Loss: 0.584 | Acc: 0.825 
[epoch:11, iter:711] Loss: 0.588 | Acc: 0.825 
[epoch:11, iter:712] Loss: 0.590 | Acc: 0.825 
[epoch:11, iter:713] Loss: 0.595 | Acc: 0.824 
[epoch:11, iter:714] Loss: 0.595 | Acc: 0.824 
[epoch:11, iter:715] Loss: 0.597 | Acc: 0.823 
[epoch:11, iter:716] Loss: 0.598 | Acc: 0.822 
[epoch:11, iter:717] Loss: 0.596 | Acc: 0.823 
[epoch:11, iter:718] Loss: 0.596 | Acc: 0.823 
[epoch:11, iter:719] Loss: 0.595 | Acc: 0.823 
[epoch:11, iter:720] Loss: 0.596 | Acc: 0.822 
[epoch:11, iter:721] Loss: 0.592 | Acc: 0.823 
[epoch:11, iter:722] Loss: 0.591 | Acc: 0.823 
[epoch:11, iter:723] Loss: 0.590 | Acc: 0.824 
[epoch:11, it

...

[epoch:58, iter:3919] Loss: 0.113 | Acc: 0.964 
[epoch:58, iter:3920] Loss: 0.113 | Acc: 0.964 
[epoch:58, iter:3921] Loss: 0.114 | Acc: 0.964 
[epoch:58, iter:3922] Loss: 0.115 | Acc: 0.963 
[epoch:58, iter:3923] Loss: 0.116 | Acc: 0.964 
[epoch:58, iter:3924] Loss: 0.116 | Acc: 0.963 
[epoch:58, iter:3925] Loss: 0.116 | Acc: 0.963 
[epoch:58, iter:3926] Loss: 0.116 | Acc: 0.963 
[epoch:58, iter:3927] Loss: 0.116 | Acc: 0.963 
[epoch:58, iter:3928] Loss: 0.115 | Acc: 0.964 
[epoch:58, iter:3929] Loss: 0.115 | Acc: 0.964 
[epoch:58, iter:3930] Loss: 0.114 | Acc: 0.964 
[epoch:58, iter:3931] Loss: 0.115 | Acc: 0.964 
[epoch:58, iter:3932] Loss: 0.115 | Acc: 0.964 
[epoch:58, iter:3933] Loss: 0.115 | Acc: 0.963 
[epoch:58, iter:3934] Loss: 0.116 | Acc: 0.963 
[epoch:58, iter:3935] Loss: 0.115 | Acc: 0.963 
[epoch:58, iter:3936] Loss: 0.117 | Acc: 0.963 
[epoch:58, iter:3937] Loss: 0.117 | Acc: 0.963 
[epoch:58, iter:3938] Loss: 0.117 | Acc: 0.963 
[epoch:58, iter:3939] Loss: 0.117 | Acc:

[epoch:61, iter:4087] Loss: 0.104 | Acc: 0.965 
[epoch:61, iter:4088] Loss: 0.103 | Acc: 0.966 
[epoch:61, iter:4089] Loss: 0.103 | Acc: 0.967 
[epoch:61, iter:4090] Loss: 0.101 | Acc: 0.968 
[epoch:61, iter:4091] Loss: 0.099 | Acc: 0.968 
[epoch:61, iter:4092] Loss: 0.101 | Acc: 0.968 
[epoch:61, iter:4093] Loss: 0.104 | Acc: 0.967 
[epoch:61, iter:4094] Loss: 0.103 | Acc: 0.968 
[epoch:61, iter:4095] Loss: 0.103 | Acc: 0.968 
[epoch:61, iter:4096] Loss: 0.101 | Acc: 0.969 
[epoch:61, iter:4097] Loss: 0.102 | Acc: 0.968 
[epoch:61, iter:4098] Loss: 0.099 | Acc: 0.969 
[epoch:61, iter:4099] Loss: 0.101 | Acc: 0.969 
[epoch:61, iter:4100] Loss: 0.103 | Acc: 0.967 
[epoch:61, iter:4101] Loss: 0.104 | Acc: 0.967 
[epoch:61, iter:4102] Loss: 0.102 | Acc: 0.968 
[epoch:61, iter:4103] Loss: 0.101 | Acc: 0.968 
[epoch:61, iter:4104] Loss: 0.101 | Acc: 0.968 
[epoch:61, iter:4105] Loss: 0.101 | Acc: 0.969 
[epoch:61, iter:4106] Loss: 0.100 | Acc: 0.969 
[epoch:61, iter:4107] Loss: 0.101 | Acc:

[epoch:63, iter:4257] Loss: 0.099 | Acc: 0.969 
[epoch:63, iter:4258] Loss: 0.100 | Acc: 0.969 
[epoch:63, iter:4259] Loss: 0.099 | Acc: 0.969 
[epoch:63, iter:4260] Loss: 0.099 | Acc: 0.969 
[epoch:63, iter:4261] Loss: 0.099 | Acc: 0.969 
[epoch:63, iter:4262] Loss: 0.100 | Acc: 0.969 
[epoch:63, iter:4263] Loss: 0.100 | Acc: 0.969 
[epoch:63, iter:4264] Loss: 0.101 | Acc: 0.969 
[epoch:63, iter:4265] Loss: 0.100 | Acc: 0.969 
[epoch:63, iter:4266] Loss: 0.101 | Acc: 0.969 
[epoch:63, iter:4267] Loss: 0.100 | Acc: 0.969 
[epoch:63, iter:4268] Loss: 0.100 | Acc: 0.969 
[epoch:63, iter:4269] Loss: 0.100 | Acc: 0.969 
[epoch:63, iter:4270] Loss: 0.100 | Acc: 0.969 
[epoch:63, iter:4271] Loss: 0.101 | Acc: 0.969 
[epoch:63, iter:4272] Loss: 0.101 | Acc: 0.969 
[epoch:63, iter:4273] Loss: 0.101 | Acc: 0.969 
[epoch:63, iter:4274] Loss: 0.101 | Acc: 0.969 
[epoch:63, iter:4275] Loss: 0.102 | Acc: 0.968 
[epoch:63, iter:4276] Loss: 0.102 | Acc: 0.968 
[epoch:63, iter:4277] Loss: 0.101 | Acc:

[epoch:66, iter:4425] Loss: 0.091 | Acc: 0.974 
[epoch:66, iter:4426] Loss: 0.091 | Acc: 0.974 
[epoch:66, iter:4427] Loss: 0.091 | Acc: 0.974 
[epoch:66, iter:4428] Loss: 0.090 | Acc: 0.976 
[epoch:66, iter:4429] Loss: 0.091 | Acc: 0.976 
[epoch:66, iter:4430] Loss: 0.096 | Acc: 0.973 
[epoch:66, iter:4431] Loss: 0.094 | Acc: 0.974 
[epoch:66, iter:4432] Loss: 0.095 | Acc: 0.973 
[epoch:66, iter:4433] Loss: 0.097 | Acc: 0.972 
[epoch:66, iter:4434] Loss: 0.093 | Acc: 0.973 
[epoch:66, iter:4435] Loss: 0.093 | Acc: 0.973 
[epoch:66, iter:4436] Loss: 0.093 | Acc: 0.972 
[epoch:66, iter:4437] Loss: 0.093 | Acc: 0.973 
[epoch:66, iter:4438] Loss: 0.092 | Acc: 0.972 
[epoch:66, iter:4439] Loss: 0.094 | Acc: 0.972 
[epoch:66, iter:4440] Loss: 0.093 | Acc: 0.972 
[epoch:66, iter:4441] Loss: 0.094 | Acc: 0.972 
[epoch:66, iter:4442] Loss: 0.095 | Acc: 0.972 
[epoch:66, iter:4443] Loss: 0.095 | Acc: 0.971 
[epoch:66, iter:4444] Loss: 0.096 | Acc: 0.970 
[epoch:66, iter:4445] Loss: 0.094 | Acc:

[epoch:68, iter:4594] Loss: 0.098 | Acc: 0.971 
[epoch:68, iter:4595] Loss: 0.098 | Acc: 0.970 
[epoch:68, iter:4596] Loss: 0.099 | Acc: 0.970 
[epoch:68, iter:4597] Loss: 0.099 | Acc: 0.970 
[epoch:68, iter:4598] Loss: 0.100 | Acc: 0.970 
[epoch:68, iter:4599] Loss: 0.099 | Acc: 0.970 
[epoch:68, iter:4600] Loss: 0.099 | Acc: 0.970 
[epoch:68, iter:4601] Loss: 0.099 | Acc: 0.970 
[epoch:68, iter:4602] Loss: 0.099 | Acc: 0.970 
[epoch:68, iter:4603] Loss: 0.100 | Acc: 0.970 
[epoch:68, iter:4604] Loss: 0.100 | Acc: 0.970 
[epoch:68, iter:4605] Loss: 0.100 | Acc: 0.970 
[epoch:68, iter:4606] Loss: 0.100 | Acc: 0.970 
[epoch:68, iter:4607] Loss: 0.100 | Acc: 0.970 
[epoch:68, iter:4608] Loss: 0.100 | Acc: 0.970 
[epoch:68, iter:4609] Loss: 0.100 | Acc: 0.970 
[epoch:68, iter:4610] Loss: 0.100 | Acc: 0.970 
[epoch:68, iter:4611] Loss: 0.099 | Acc: 0.970 
[epoch:68, iter:4612] Loss: 0.100 | Acc: 0.969 
[epoch:68, iter:4613] Loss: 0.100 | Acc: 0.969 
[epoch:68, iter:4614] Loss: 0.100 | Acc:

In [30]:
net2_new = train_sch(2)
val2_pred = pred_val(2)

Start Training, resnet!

Epoch: 1
[epoch:1, iter:1] Loss: 3.372 | Acc: 0.004 
[epoch:1, iter:2] Loss: 5.583 | Acc: 0.182 
[epoch:1, iter:3] Loss: 6.161 | Acc: 0.186 
[epoch:1, iter:4] Loss: 6.002 | Acc: 0.184 
[epoch:1, iter:5] Loss: 5.421 | Acc: 0.162 
[epoch:1, iter:6] Loss: 4.984 | Acc: 0.197 
[epoch:1, iter:7] Loss: 4.680 | Acc: 0.220 
[epoch:1, iter:8] Loss: 4.436 | Acc: 0.238 
[epoch:1, iter:9] Loss: 4.239 | Acc: 0.253 
[epoch:1, iter:10] Loss: 4.085 | Acc: 0.262 
[epoch:1, iter:11] Loss: 3.944 | Acc: 0.271 
[epoch:1, iter:12] Loss: 3.824 | Acc: 0.280 
[epoch:1, iter:13] Loss: 3.725 | Acc: 0.284 
[epoch:1, iter:14] Loss: 3.628 | Acc: 0.288 
[epoch:1, iter:15] Loss: 3.539 | Acc: 0.296 
[epoch:1, iter:16] Loss: 3.458 | Acc: 0.303 
[epoch:1, iter:17] Loss: 3.387 | Acc: 0.308 
[epoch:1, iter:18] Loss: 3.336 | Acc: 0.314 
[epoch:1, iter:19] Loss: 3.285 | Acc: 0.318 
[epoch:1, iter:20] Loss: 3.239 | Acc: 0.321 
[epoch:1, iter:21] Loss: 3.189 | Acc: 0.325 
[epoch:1, iter:22] Loss: 3.148

[epoch:3, iter:179] Loss: 1.503 | Acc: 0.582 
[epoch:3, iter:180] Loss: 1.503 | Acc: 0.582 
[epoch:3, iter:181] Loss: 1.504 | Acc: 0.581 
[epoch:3, iter:182] Loss: 1.503 | Acc: 0.582 
[epoch:3, iter:183] Loss: 1.502 | Acc: 0.582 
[epoch:3, iter:184] Loss: 1.498 | Acc: 0.583 
[epoch:3, iter:185] Loss: 1.497 | Acc: 0.584 
[epoch:3, iter:186] Loss: 1.495 | Acc: 0.584 
[epoch:3, iter:187] Loss: 1.494 | Acc: 0.584 
[epoch:3, iter:188] Loss: 1.490 | Acc: 0.585 
[epoch:3, iter:189] Loss: 1.488 | Acc: 0.586 
[epoch:3, iter:190] Loss: 1.485 | Acc: 0.586 
[epoch:3, iter:191] Loss: 1.483 | Acc: 0.586 
[epoch:3, iter:192] Loss: 1.479 | Acc: 0.587 
[epoch:3, iter:193] Loss: 1.478 | Acc: 0.588 
[epoch:3, iter:194] Loss: 1.479 | Acc: 0.587 
[epoch:3, iter:195] Loss: 1.480 | Acc: 0.587 
[epoch:3, iter:196] Loss: 1.478 | Acc: 0.588 
[epoch:3, iter:197] Loss: 1.479 | Acc: 0.587 
[epoch:3, iter:198] Loss: 1.479 | Acc: 0.586 
[epoch:3, iter:199] Loss: 1.481 | Acc: 0.586 
[epoch:3, iter:200] Loss: 1.478 | 

[epoch:6, iter:354] Loss: 0.998 | Acc: 0.712 
[epoch:6, iter:355] Loss: 0.997 | Acc: 0.713 
[epoch:6, iter:356] Loss: 0.996 | Acc: 0.714 
[epoch:6, iter:357] Loss: 0.996 | Acc: 0.713 
[epoch:6, iter:358] Loss: 0.989 | Acc: 0.716 
[epoch:6, iter:359] Loss: 0.985 | Acc: 0.715 
[epoch:6, iter:360] Loss: 0.977 | Acc: 0.717 
[epoch:6, iter:361] Loss: 0.975 | Acc: 0.715 
[epoch:6, iter:362] Loss: 0.979 | Acc: 0.713 
[epoch:6, iter:363] Loss: 0.975 | Acc: 0.715 
[epoch:6, iter:364] Loss: 0.978 | Acc: 0.714 
[epoch:6, iter:365] Loss: 0.969 | Acc: 0.716 
[epoch:6, iter:366] Loss: 0.969 | Acc: 0.716 
[epoch:6, iter:367] Loss: 0.969 | Acc: 0.715 
[epoch:6, iter:368] Loss: 0.971 | Acc: 0.715 
[epoch:6, iter:369] Loss: 0.965 | Acc: 0.717 
[epoch:6, iter:370] Loss: 0.966 | Acc: 0.718 
[epoch:6, iter:371] Loss: 0.964 | Acc: 0.718 
[epoch:6, iter:372] Loss: 0.966 | Acc: 0.719 
[epoch:6, iter:373] Loss: 0.961 | Acc: 0.720 
[epoch:6, iter:374] Loss: 0.959 | Acc: 0.721 
[epoch:6, iter:375] Loss: 0.961 | 

[epoch:8, iter:530] Loss: 0.785 | Acc: 0.774 
[epoch:8, iter:531] Loss: 0.783 | Acc: 0.774 
[epoch:8, iter:532] Loss: 0.782 | Acc: 0.774 
[epoch:8, iter:533] Loss: 0.780 | Acc: 0.775 
[epoch:8, iter:534] Loss: 0.778 | Acc: 0.775 
[epoch:8, iter:535] Loss: 0.776 | Acc: 0.776 
[epoch:8, iter:536] Loss: 0.776 | Acc: 0.776 
[epoch:8, iter:537] Loss: 0.776 | Acc: 0.776 
[epoch:8, iter:538] Loss: 0.775 | Acc: 0.777 
[epoch:8, iter:539] Loss: 0.776 | Acc: 0.776 
[epoch:8, iter:540] Loss: 0.775 | Acc: 0.776 
[epoch:8, iter:541] Loss: 0.776 | Acc: 0.776 
[epoch:8, iter:542] Loss: 0.774 | Acc: 0.776 
[epoch:8, iter:543] Loss: 0.775 | Acc: 0.776 
[epoch:8, iter:544] Loss: 0.774 | Acc: 0.776 
Waiting Test!
val acc：0.767
Saving model......

Epoch: 9
[epoch:9, iter:545] Loss: 0.715 | Acc: 0.777 
[epoch:9, iter:546] Loss: 0.708 | Acc: 0.787 
[epoch:9, iter:547] Loss: 0.735 | Acc: 0.788 
[epoch:9, iter:548] Loss: 0.720 | Acc: 0.787 
[epoch:9, iter:549] Loss: 0.738 | Acc: 0.783 
[epoch:9, iter:550] Los

[epoch:11, iter:703] Loss: 0.632 | Acc: 0.814 
[epoch:11, iter:704] Loss: 0.629 | Acc: 0.815 
[epoch:11, iter:705] Loss: 0.633 | Acc: 0.813 
[epoch:11, iter:706] Loss: 0.635 | Acc: 0.812 
[epoch:11, iter:707] Loss: 0.634 | Acc: 0.813 
[epoch:11, iter:708] Loss: 0.633 | Acc: 0.813 
[epoch:11, iter:709] Loss: 0.634 | Acc: 0.813 
[epoch:11, iter:710] Loss: 0.634 | Acc: 0.813 
[epoch:11, iter:711] Loss: 0.632 | Acc: 0.813 
[epoch:11, iter:712] Loss: 0.632 | Acc: 0.813 
[epoch:11, iter:713] Loss: 0.630 | Acc: 0.814 
[epoch:11, iter:714] Loss: 0.637 | Acc: 0.813 
[epoch:11, iter:715] Loss: 0.636 | Acc: 0.813 
[epoch:11, iter:716] Loss: 0.633 | Acc: 0.814 
[epoch:11, iter:717] Loss: 0.635 | Acc: 0.814 
[epoch:11, iter:718] Loss: 0.633 | Acc: 0.814 
[epoch:11, iter:719] Loss: 0.633 | Acc: 0.815 
[epoch:11, iter:720] Loss: 0.636 | Acc: 0.814 
[epoch:11, iter:721] Loss: 0.635 | Acc: 0.815 
[epoch:11, iter:722] Loss: 0.635 | Acc: 0.814 
[epoch:11, iter:723] Loss: 0.634 | Acc: 0.814 
[epoch:11, it

...

[epoch:58, iter:3918] Loss: 0.124 | Acc: 0.963 
[epoch:58, iter:3919] Loss: 0.125 | Acc: 0.963 
[epoch:58, iter:3920] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3921] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3922] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3923] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3924] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3925] Loss: 0.124 | Acc: 0.962 
[epoch:58, iter:3926] Loss: 0.124 | Acc: 0.962 
[epoch:58, iter:3927] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3928] Loss: 0.126 | Acc: 0.962 
[epoch:58, iter:3929] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3930] Loss: 0.126 | Acc: 0.962 
[epoch:58, iter:3931] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3932] Loss: 0.126 | Acc: 0.962 
[epoch:58, iter:3933] Loss: 0.126 | Acc: 0.962 
[epoch:58, iter:3934] Loss: 0.127 | Acc: 0.961 
[epoch:58, iter:3935] Loss: 0.127 | Acc: 0.961 
[epoch:58, iter:3936] Loss: 0.127 | Acc: 0.961 
[epoch:58, iter:3937] Loss: 0.127 | Acc: 0.961 
[epoch:58, iter:3938] Loss: 0.127 | Acc:

[epoch:61, iter:4087] Loss: 0.110 | Acc: 0.968 
[epoch:61, iter:4088] Loss: 0.112 | Acc: 0.968 
[epoch:61, iter:4089] Loss: 0.110 | Acc: 0.968 
[epoch:61, iter:4090] Loss: 0.110 | Acc: 0.967 
[epoch:61, iter:4091] Loss: 0.110 | Acc: 0.966 
[epoch:61, iter:4092] Loss: 0.107 | Acc: 0.967 
[epoch:61, iter:4093] Loss: 0.111 | Acc: 0.966 
[epoch:61, iter:4094] Loss: 0.110 | Acc: 0.967 
[epoch:61, iter:4095] Loss: 0.112 | Acc: 0.967 
[epoch:61, iter:4096] Loss: 0.111 | Acc: 0.967 
[epoch:61, iter:4097] Loss: 0.109 | Acc: 0.968 
[epoch:61, iter:4098] Loss: 0.108 | Acc: 0.967 
[epoch:61, iter:4099] Loss: 0.108 | Acc: 0.967 
[epoch:61, iter:4100] Loss: 0.109 | Acc: 0.967 
[epoch:61, iter:4101] Loss: 0.109 | Acc: 0.967 
[epoch:61, iter:4102] Loss: 0.108 | Acc: 0.967 
[epoch:61, iter:4103] Loss: 0.107 | Acc: 0.967 
[epoch:61, iter:4104] Loss: 0.109 | Acc: 0.966 
[epoch:61, iter:4105] Loss: 0.109 | Acc: 0.966 
[epoch:61, iter:4106] Loss: 0.109 | Acc: 0.966 
[epoch:61, iter:4107] Loss: 0.109 | Acc:

[epoch:63, iter:4256] Loss: 0.106 | Acc: 0.968 
[epoch:63, iter:4257] Loss: 0.105 | Acc: 0.968 
[epoch:63, iter:4258] Loss: 0.105 | Acc: 0.968 
[epoch:63, iter:4259] Loss: 0.105 | Acc: 0.968 
[epoch:63, iter:4260] Loss: 0.105 | Acc: 0.968 
[epoch:63, iter:4261] Loss: 0.105 | Acc: 0.968 
[epoch:63, iter:4262] Loss: 0.107 | Acc: 0.967 
[epoch:63, iter:4263] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4264] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4265] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4266] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4267] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4268] Loss: 0.105 | Acc: 0.967 
[epoch:63, iter:4269] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4270] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4271] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4272] Loss: 0.105 | Acc: 0.967 
[epoch:63, iter:4273] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4274] Loss: 0.105 | Acc: 0.967 
[epoch:63, iter:4275] Loss: 0.105 | Acc: 0.967 
[epoch:63, iter:4276] Loss: 0.105 | Acc:

[epoch:66, iter:4425] Loss: 0.140 | Acc: 0.959 
[epoch:66, iter:4426] Loss: 0.140 | Acc: 0.959 
[epoch:66, iter:4427] Loss: 0.132 | Acc: 0.962 
[epoch:66, iter:4428] Loss: 0.131 | Acc: 0.962 
[epoch:66, iter:4429] Loss: 0.128 | Acc: 0.964 
[epoch:66, iter:4430] Loss: 0.129 | Acc: 0.961 
[epoch:66, iter:4431] Loss: 0.126 | Acc: 0.962 
[epoch:66, iter:4432] Loss: 0.125 | Acc: 0.963 
[epoch:66, iter:4433] Loss: 0.122 | Acc: 0.964 
[epoch:66, iter:4434] Loss: 0.119 | Acc: 0.965 
[epoch:66, iter:4435] Loss: 0.117 | Acc: 0.966 
[epoch:66, iter:4436] Loss: 0.118 | Acc: 0.966 
[epoch:66, iter:4437] Loss: 0.115 | Acc: 0.966 
[epoch:66, iter:4438] Loss: 0.113 | Acc: 0.967 
[epoch:66, iter:4439] Loss: 0.114 | Acc: 0.966 
[epoch:66, iter:4440] Loss: 0.112 | Acc: 0.967 
[epoch:66, iter:4441] Loss: 0.112 | Acc: 0.967 
[epoch:66, iter:4442] Loss: 0.110 | Acc: 0.967 
[epoch:66, iter:4443] Loss: 0.110 | Acc: 0.967 
[epoch:66, iter:4444] Loss: 0.111 | Acc: 0.967 
[epoch:66, iter:4445] Loss: 0.110 | Acc:

[epoch:68, iter:4595] Loss: 0.106 | Acc: 0.969 
[epoch:68, iter:4596] Loss: 0.106 | Acc: 0.968 
[epoch:68, iter:4597] Loss: 0.107 | Acc: 0.968 
[epoch:68, iter:4598] Loss: 0.106 | Acc: 0.968 
[epoch:68, iter:4599] Loss: 0.106 | Acc: 0.968 
[epoch:68, iter:4600] Loss: 0.106 | Acc: 0.968 
[epoch:68, iter:4601] Loss: 0.105 | Acc: 0.969 
[epoch:68, iter:4602] Loss: 0.105 | Acc: 0.968 
[epoch:68, iter:4603] Loss: 0.105 | Acc: 0.968 
[epoch:68, iter:4604] Loss: 0.105 | Acc: 0.968 
[epoch:68, iter:4605] Loss: 0.106 | Acc: 0.968 
[epoch:68, iter:4606] Loss: 0.106 | Acc: 0.968 
[epoch:68, iter:4607] Loss: 0.106 | Acc: 0.967 
[epoch:68, iter:4608] Loss: 0.107 | Acc: 0.967 
[epoch:68, iter:4609] Loss: 0.107 | Acc: 0.967 
[epoch:68, iter:4610] Loss: 0.108 | Acc: 0.967 
[epoch:68, iter:4611] Loss: 0.108 | Acc: 0.967 
[epoch:68, iter:4612] Loss: 0.109 | Acc: 0.967 
[epoch:68, iter:4613] Loss: 0.108 | Acc: 0.967 
[epoch:68, iter:4614] Loss: 0.108 | Acc: 0.967 
[epoch:68, iter:4615] Loss: 0.109 | Acc:

In [19]:
val3_pred = pred_val(3)

val acc：0.921


In [17]:
net3_new = train_sch(3)
# val3_pred = pred_val(3)

Start Training, resnet!

Epoch: 1
[epoch:1, iter:1] Loss: 3.513 | Acc: 0.008 
[epoch:1, iter:2] Loss: 6.520 | Acc: 0.158 
[epoch:1, iter:3] Loss: 6.258 | Acc: 0.158 
[epoch:1, iter:4] Loss: 5.551 | Acc: 0.155 
[epoch:1, iter:5] Loss: 5.094 | Acc: 0.138 
[epoch:1, iter:6] Loss: 4.771 | Acc: 0.124 
[epoch:1, iter:7] Loss: 4.495 | Acc: 0.114 
[epoch:1, iter:8] Loss: 4.261 | Acc: 0.138 
[epoch:1, iter:9] Loss: 4.076 | Acc: 0.163 
[epoch:1, iter:10] Loss: 3.954 | Acc: 0.179 
[epoch:1, iter:11] Loss: 3.823 | Acc: 0.192 
[epoch:1, iter:12] Loss: 3.711 | Acc: 0.199 
[epoch:1, iter:13] Loss: 3.613 | Acc: 0.199 
[epoch:1, iter:14] Loss: 3.533 | Acc: 0.199 
[epoch:1, iter:15] Loss: 3.466 | Acc: 0.202 
[epoch:1, iter:16] Loss: 3.392 | Acc: 0.216 
[epoch:1, iter:17] Loss: 3.337 | Acc: 0.225 
[epoch:1, iter:18] Loss: 3.293 | Acc: 0.233 
[epoch:1, iter:19] Loss: 3.247 | Acc: 0.240 
[epoch:1, iter:20] Loss: 3.202 | Acc: 0.246 
[epoch:1, iter:21] Loss: 3.160 | Acc: 0.251 
[epoch:1, iter:22] Loss: 3.123

[epoch:3, iter:179] Loss: 1.602 | Acc: 0.542 
[epoch:3, iter:180] Loss: 1.594 | Acc: 0.544 
[epoch:3, iter:181] Loss: 1.590 | Acc: 0.545 
[epoch:3, iter:182] Loss: 1.588 | Acc: 0.545 
[epoch:3, iter:183] Loss: 1.581 | Acc: 0.548 
[epoch:3, iter:184] Loss: 1.575 | Acc: 0.550 
[epoch:3, iter:185] Loss: 1.571 | Acc: 0.551 
[epoch:3, iter:186] Loss: 1.569 | Acc: 0.552 
[epoch:3, iter:187] Loss: 1.564 | Acc: 0.553 
[epoch:3, iter:188] Loss: 1.560 | Acc: 0.554 
[epoch:3, iter:189] Loss: 1.555 | Acc: 0.556 
[epoch:3, iter:190] Loss: 1.550 | Acc: 0.556 
[epoch:3, iter:191] Loss: 1.549 | Acc: 0.557 
[epoch:3, iter:192] Loss: 1.547 | Acc: 0.558 
[epoch:3, iter:193] Loss: 1.545 | Acc: 0.558 
[epoch:3, iter:194] Loss: 1.543 | Acc: 0.559 
[epoch:3, iter:195] Loss: 1.542 | Acc: 0.560 
[epoch:3, iter:196] Loss: 1.540 | Acc: 0.561 
[epoch:3, iter:197] Loss: 1.535 | Acc: 0.562 
[epoch:3, iter:198] Loss: 1.533 | Acc: 0.562 
[epoch:3, iter:199] Loss: 1.532 | Acc: 0.563 
[epoch:3, iter:200] Loss: 1.530 | 

[epoch:6, iter:353] Loss: 1.005 | Acc: 0.717 
[epoch:6, iter:354] Loss: 0.998 | Acc: 0.718 
[epoch:6, iter:355] Loss: 0.999 | Acc: 0.716 
[epoch:6, iter:356] Loss: 0.990 | Acc: 0.718 
[epoch:6, iter:357] Loss: 0.991 | Acc: 0.719 
[epoch:6, iter:358] Loss: 0.995 | Acc: 0.720 
[epoch:6, iter:359] Loss: 0.990 | Acc: 0.723 
[epoch:6, iter:360] Loss: 0.988 | Acc: 0.722 
[epoch:6, iter:361] Loss: 0.984 | Acc: 0.723 
[epoch:6, iter:362] Loss: 0.979 | Acc: 0.723 
[epoch:6, iter:363] Loss: 0.987 | Acc: 0.720 
[epoch:6, iter:364] Loss: 0.987 | Acc: 0.721 
[epoch:6, iter:365] Loss: 0.986 | Acc: 0.720 
[epoch:6, iter:366] Loss: 0.981 | Acc: 0.722 
[epoch:6, iter:367] Loss: 0.978 | Acc: 0.722 
[epoch:6, iter:368] Loss: 0.974 | Acc: 0.724 
[epoch:6, iter:369] Loss: 0.972 | Acc: 0.724 
[epoch:6, iter:370] Loss: 0.971 | Acc: 0.724 
[epoch:6, iter:371] Loss: 0.970 | Acc: 0.724 
[epoch:6, iter:372] Loss: 0.970 | Acc: 0.725 
[epoch:6, iter:373] Loss: 0.968 | Acc: 0.726 
[epoch:6, iter:374] Loss: 0.969 | 

[epoch:8, iter:530] Loss: 0.809 | Acc: 0.765 
[epoch:8, iter:531] Loss: 0.807 | Acc: 0.766 
[epoch:8, iter:532] Loss: 0.806 | Acc: 0.766 
[epoch:8, iter:533] Loss: 0.806 | Acc: 0.766 
[epoch:8, iter:534] Loss: 0.805 | Acc: 0.766 
[epoch:8, iter:535] Loss: 0.803 | Acc: 0.767 
[epoch:8, iter:536] Loss: 0.802 | Acc: 0.767 
[epoch:8, iter:537] Loss: 0.802 | Acc: 0.766 
[epoch:8, iter:538] Loss: 0.802 | Acc: 0.767 
[epoch:8, iter:539] Loss: 0.799 | Acc: 0.767 
[epoch:8, iter:540] Loss: 0.799 | Acc: 0.767 
[epoch:8, iter:541] Loss: 0.798 | Acc: 0.768 
[epoch:8, iter:542] Loss: 0.797 | Acc: 0.768 
[epoch:8, iter:543] Loss: 0.798 | Acc: 0.768 
[epoch:8, iter:544] Loss: 0.797 | Acc: 0.768 
Waiting Test!
val acc：0.798
Saving model......

Epoch: 9
[epoch:9, iter:545] Loss: 0.744 | Acc: 0.785 
[epoch:9, iter:546] Loss: 0.732 | Acc: 0.803 
[epoch:9, iter:547] Loss: 0.742 | Acc: 0.803 
[epoch:9, iter:548] Loss: 0.735 | Acc: 0.798 
[epoch:9, iter:549] Loss: 0.740 | Acc: 0.795 
[epoch:9, iter:550] Los

[epoch:11, iter:703] Loss: 0.662 | Acc: 0.801 
[epoch:11, iter:704] Loss: 0.663 | Acc: 0.801 
[epoch:11, iter:705] Loss: 0.659 | Acc: 0.803 
[epoch:11, iter:706] Loss: 0.661 | Acc: 0.802 
[epoch:11, iter:707] Loss: 0.663 | Acc: 0.802 
[epoch:11, iter:708] Loss: 0.663 | Acc: 0.801 
[epoch:11, iter:709] Loss: 0.660 | Acc: 0.803 
[epoch:11, iter:710] Loss: 0.658 | Acc: 0.804 
[epoch:11, iter:711] Loss: 0.660 | Acc: 0.804 
[epoch:11, iter:712] Loss: 0.666 | Acc: 0.803 
[epoch:11, iter:713] Loss: 0.665 | Acc: 0.803 
[epoch:11, iter:714] Loss: 0.666 | Acc: 0.803 
[epoch:11, iter:715] Loss: 0.665 | Acc: 0.804 
[epoch:11, iter:716] Loss: 0.663 | Acc: 0.804 
[epoch:11, iter:717] Loss: 0.662 | Acc: 0.805 
[epoch:11, iter:718] Loss: 0.667 | Acc: 0.804 
[epoch:11, iter:719] Loss: 0.666 | Acc: 0.804 
[epoch:11, iter:720] Loss: 0.665 | Acc: 0.804 
[epoch:11, iter:721] Loss: 0.665 | Acc: 0.804 
[epoch:11, iter:722] Loss: 0.664 | Acc: 0.805 
[epoch:11, iter:723] Loss: 0.663 | Acc: 0.805 
[epoch:11, it

...

[epoch:61, iter:4088] Loss: 0.114 | Acc: 0.967 
[epoch:61, iter:4089] Loss: 0.111 | Acc: 0.967 
[epoch:61, iter:4090] Loss: 0.109 | Acc: 0.968 
[epoch:61, iter:4091] Loss: 0.111 | Acc: 0.968 
[epoch:61, iter:4092] Loss: 0.112 | Acc: 0.968 
[epoch:61, iter:4093] Loss: 0.111 | Acc: 0.968 
[epoch:61, iter:4094] Loss: 0.113 | Acc: 0.967 
[epoch:61, iter:4095] Loss: 0.113 | Acc: 0.966 
[epoch:61, iter:4096] Loss: 0.115 | Acc: 0.966 
[epoch:61, iter:4097] Loss: 0.115 | Acc: 0.965 
[epoch:61, iter:4098] Loss: 0.118 | Acc: 0.965 
[epoch:61, iter:4099] Loss: 0.120 | Acc: 0.964 
[epoch:61, iter:4100] Loss: 0.118 | Acc: 0.965 
[epoch:61, iter:4101] Loss: 0.117 | Acc: 0.965 
[epoch:61, iter:4102] Loss: 0.117 | Acc: 0.964 
[epoch:61, iter:4103] Loss: 0.116 | Acc: 0.965 
[epoch:61, iter:4104] Loss: 0.115 | Acc: 0.965 
[epoch:61, iter:4105] Loss: 0.117 | Acc: 0.964 
[epoch:61, iter:4106] Loss: 0.116 | Acc: 0.965 
[epoch:61, iter:4107] Loss: 0.114 | Acc: 0.965 
[epoch:61, iter:4108] Loss: 0.113 | Acc:

[epoch:63, iter:4258] Loss: 0.108 | Acc: 0.964 
[epoch:63, iter:4259] Loss: 0.108 | Acc: 0.964 
[epoch:63, iter:4260] Loss: 0.107 | Acc: 0.964 
[epoch:63, iter:4261] Loss: 0.108 | Acc: 0.964 
[epoch:63, iter:4262] Loss: 0.108 | Acc: 0.965 
[epoch:63, iter:4263] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4264] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4265] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4266] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4267] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4268] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4269] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4270] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4271] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4272] Loss: 0.107 | Acc: 0.964 
[epoch:63, iter:4273] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4274] Loss: 0.107 | Acc: 0.965 
[epoch:63, iter:4275] Loss: 0.106 | Acc: 0.965 
[epoch:63, iter:4276] Loss: 0.106 | Acc: 0.965 
[epoch:63, iter:4277] Loss: 0.106 | Acc: 0.966 
[epoch:63, iter:4278] Loss: 0.106 | Acc:

[epoch:66, iter:4427] Loss: 0.100 | Acc: 0.970 
[epoch:66, iter:4428] Loss: 0.099 | Acc: 0.970 
[epoch:66, iter:4429] Loss: 0.098 | Acc: 0.972 
[epoch:66, iter:4430] Loss: 0.097 | Acc: 0.972 
[epoch:66, iter:4431] Loss: 0.096 | Acc: 0.973 
[epoch:66, iter:4432] Loss: 0.096 | Acc: 0.974 
[epoch:66, iter:4433] Loss: 0.097 | Acc: 0.973 
[epoch:66, iter:4434] Loss: 0.096 | Acc: 0.973 
[epoch:66, iter:4435] Loss: 0.096 | Acc: 0.974 
[epoch:66, iter:4436] Loss: 0.096 | Acc: 0.973 
[epoch:66, iter:4437] Loss: 0.097 | Acc: 0.973 
[epoch:66, iter:4438] Loss: 0.097 | Acc: 0.973 
[epoch:66, iter:4439] Loss: 0.097 | Acc: 0.972 
[epoch:66, iter:4440] Loss: 0.097 | Acc: 0.972 
[epoch:66, iter:4441] Loss: 0.098 | Acc: 0.972 
[epoch:66, iter:4442] Loss: 0.100 | Acc: 0.971 
[epoch:66, iter:4443] Loss: 0.100 | Acc: 0.971 
[epoch:66, iter:4444] Loss: 0.100 | Acc: 0.971 
[epoch:66, iter:4445] Loss: 0.099 | Acc: 0.971 
[epoch:66, iter:4446] Loss: 0.100 | Acc: 0.971 
[epoch:66, iter:4447] Loss: 0.102 | Acc:

[epoch:68, iter:4597] Loss: 0.103 | Acc: 0.969 
[epoch:68, iter:4598] Loss: 0.103 | Acc: 0.969 
[epoch:68, iter:4599] Loss: 0.103 | Acc: 0.969 
[epoch:68, iter:4600] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4601] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4602] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4603] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4604] Loss: 0.103 | Acc: 0.969 
[epoch:68, iter:4605] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4606] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4607] Loss: 0.101 | Acc: 0.969 
[epoch:68, iter:4608] Loss: 0.101 | Acc: 0.969 
[epoch:68, iter:4609] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4610] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4611] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4612] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4613] Loss: 0.102 | Acc: 0.968 
[epoch:68, iter:4614] Loss: 0.103 | Acc: 0.968 
[epoch:68, iter:4615] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4616] Loss: 0.102 | Acc: 0.968 
[epoch:68, iter:4617] Loss: 0.103 | Acc:

NameError: name 'pred_val' is not defined

In [20]:
net4_new = train_sch(4)
val4_pred = pred_val(4)

Start Training, resnet!

Epoch: 1
[epoch:1, iter:1] Loss: 3.354 | Acc: 0.023 
[epoch:1, iter:2] Loss: 5.667 | Acc: 0.211 
[epoch:1, iter:3] Loss: 5.755 | Acc: 0.204 
[epoch:1, iter:4] Loss: 5.171 | Acc: 0.171 
[epoch:1, iter:5] Loss: 4.753 | Acc: 0.145 
[epoch:1, iter:6] Loss: 4.444 | Acc: 0.168 
[epoch:1, iter:7] Loss: 4.170 | Acc: 0.200 
[epoch:1, iter:8] Loss: 3.973 | Acc: 0.221 
[epoch:1, iter:9] Loss: 3.784 | Acc: 0.243 
[epoch:1, iter:10] Loss: 3.629 | Acc: 0.256 
[epoch:1, iter:11] Loss: 3.516 | Acc: 0.270 
[epoch:1, iter:12] Loss: 3.425 | Acc: 0.278 
[epoch:1, iter:13] Loss: 3.340 | Acc: 0.284 
[epoch:1, iter:14] Loss: 3.274 | Acc: 0.292 
[epoch:1, iter:15] Loss: 3.213 | Acc: 0.297 
[epoch:1, iter:16] Loss: 3.144 | Acc: 0.306 
[epoch:1, iter:17] Loss: 3.095 | Acc: 0.308 
[epoch:1, iter:18] Loss: 3.051 | Acc: 0.313 
[epoch:1, iter:19] Loss: 2.996 | Acc: 0.321 
[epoch:1, iter:20] Loss: 2.957 | Acc: 0.327 
[epoch:1, iter:21] Loss: 2.931 | Acc: 0.330 
[epoch:1, iter:22] Loss: 2.901

[epoch:3, iter:179] Loss: 1.455 | Acc: 0.587 
[epoch:3, iter:180] Loss: 1.457 | Acc: 0.587 
[epoch:3, iter:181] Loss: 1.454 | Acc: 0.588 
[epoch:3, iter:182] Loss: 1.453 | Acc: 0.589 
[epoch:3, iter:183] Loss: 1.451 | Acc: 0.589 
[epoch:3, iter:184] Loss: 1.454 | Acc: 0.588 
[epoch:3, iter:185] Loss: 1.450 | Acc: 0.589 
[epoch:3, iter:186] Loss: 1.448 | Acc: 0.590 
[epoch:3, iter:187] Loss: 1.445 | Acc: 0.590 
[epoch:3, iter:188] Loss: 1.441 | Acc: 0.591 
[epoch:3, iter:189] Loss: 1.438 | Acc: 0.592 
[epoch:3, iter:190] Loss: 1.433 | Acc: 0.593 
[epoch:3, iter:191] Loss: 1.431 | Acc: 0.594 
[epoch:3, iter:192] Loss: 1.429 | Acc: 0.594 
[epoch:3, iter:193] Loss: 1.427 | Acc: 0.594 
[epoch:3, iter:194] Loss: 1.425 | Acc: 0.595 
[epoch:3, iter:195] Loss: 1.425 | Acc: 0.595 
[epoch:3, iter:196] Loss: 1.422 | Acc: 0.596 
[epoch:3, iter:197] Loss: 1.423 | Acc: 0.596 
[epoch:3, iter:198] Loss: 1.424 | Acc: 0.596 
[epoch:3, iter:199] Loss: 1.422 | Acc: 0.596 
[epoch:3, iter:200] Loss: 1.422 | 

[epoch:6, iter:354] Loss: 0.941 | Acc: 0.730 
[epoch:6, iter:355] Loss: 0.940 | Acc: 0.730 
[epoch:6, iter:356] Loss: 0.939 | Acc: 0.731 
[epoch:6, iter:357] Loss: 0.931 | Acc: 0.732 
[epoch:6, iter:358] Loss: 0.928 | Acc: 0.734 
[epoch:6, iter:359] Loss: 0.931 | Acc: 0.733 
[epoch:6, iter:360] Loss: 0.933 | Acc: 0.733 
[epoch:6, iter:361] Loss: 0.930 | Acc: 0.734 
[epoch:6, iter:362] Loss: 0.936 | Acc: 0.733 
[epoch:6, iter:363] Loss: 0.939 | Acc: 0.732 
[epoch:6, iter:364] Loss: 0.936 | Acc: 0.732 
[epoch:6, iter:365] Loss: 0.934 | Acc: 0.732 
[epoch:6, iter:366] Loss: 0.931 | Acc: 0.733 
[epoch:6, iter:367] Loss: 0.935 | Acc: 0.733 
[epoch:6, iter:368] Loss: 0.937 | Acc: 0.733 
[epoch:6, iter:369] Loss: 0.935 | Acc: 0.733 
[epoch:6, iter:370] Loss: 0.937 | Acc: 0.734 
[epoch:6, iter:371] Loss: 0.938 | Acc: 0.734 
[epoch:6, iter:372] Loss: 0.936 | Acc: 0.734 
[epoch:6, iter:373] Loss: 0.935 | Acc: 0.733 
[epoch:6, iter:374] Loss: 0.939 | Acc: 0.731 
[epoch:6, iter:375] Loss: 0.939 | 

[epoch:8, iter:531] Loss: 0.744 | Acc: 0.784 
[epoch:8, iter:532] Loss: 0.746 | Acc: 0.784 
[epoch:8, iter:533] Loss: 0.746 | Acc: 0.783 
[epoch:8, iter:534] Loss: 0.748 | Acc: 0.783 
[epoch:8, iter:535] Loss: 0.747 | Acc: 0.783 
[epoch:8, iter:536] Loss: 0.748 | Acc: 0.782 
[epoch:8, iter:537] Loss: 0.747 | Acc: 0.783 
[epoch:8, iter:538] Loss: 0.746 | Acc: 0.783 
[epoch:8, iter:539] Loss: 0.746 | Acc: 0.783 
[epoch:8, iter:540] Loss: 0.747 | Acc: 0.782 
[epoch:8, iter:541] Loss: 0.748 | Acc: 0.782 
[epoch:8, iter:542] Loss: 0.747 | Acc: 0.782 
[epoch:8, iter:543] Loss: 0.747 | Acc: 0.782 
[epoch:8, iter:544] Loss: 0.746 | Acc: 0.782 
Waiting Test!
val acc：0.586
Saving model......

Epoch: 9
[epoch:9, iter:545] Loss: 0.570 | Acc: 0.855 
[epoch:9, iter:546] Loss: 0.635 | Acc: 0.824 
[epoch:9, iter:547] Loss: 0.673 | Acc: 0.809 
[epoch:9, iter:548] Loss: 0.703 | Acc: 0.794 
[epoch:9, iter:549] Loss: 0.682 | Acc: 0.804 
[epoch:9, iter:550] Loss: 0.693 | Acc: 0.798 
[epoch:9, iter:551] Los

[epoch:11, iter:703] Loss: 0.631 | Acc: 0.816 
[epoch:11, iter:704] Loss: 0.632 | Acc: 0.815 
[epoch:11, iter:705] Loss: 0.632 | Acc: 0.815 
[epoch:11, iter:706] Loss: 0.631 | Acc: 0.816 
[epoch:11, iter:707] Loss: 0.631 | Acc: 0.815 
[epoch:11, iter:708] Loss: 0.628 | Acc: 0.817 
[epoch:11, iter:709] Loss: 0.629 | Acc: 0.816 
[epoch:11, iter:710] Loss: 0.628 | Acc: 0.817 
[epoch:11, iter:711] Loss: 0.625 | Acc: 0.818 
[epoch:11, iter:712] Loss: 0.625 | Acc: 0.816 
[epoch:11, iter:713] Loss: 0.625 | Acc: 0.816 
[epoch:11, iter:714] Loss: 0.629 | Acc: 0.815 
[epoch:11, iter:715] Loss: 0.629 | Acc: 0.814 
[epoch:11, iter:716] Loss: 0.628 | Acc: 0.815 
[epoch:11, iter:717] Loss: 0.624 | Acc: 0.816 
[epoch:11, iter:718] Loss: 0.621 | Acc: 0.817 
[epoch:11, iter:719] Loss: 0.621 | Acc: 0.817 
[epoch:11, iter:720] Loss: 0.615 | Acc: 0.819 
[epoch:11, iter:721] Loss: 0.614 | Acc: 0.819 
[epoch:11, iter:722] Loss: 0.612 | Acc: 0.819 
[epoch:11, iter:723] Loss: 0.608 | Acc: 0.820 
[epoch:11, it

...

[epoch:61, iter:4086] Loss: 0.108 | Acc: 0.965 
[epoch:61, iter:4087] Loss: 0.107 | Acc: 0.965 
[epoch:61, iter:4088] Loss: 0.103 | Acc: 0.967 
[epoch:61, iter:4089] Loss: 0.107 | Acc: 0.966 
[epoch:61, iter:4090] Loss: 0.105 | Acc: 0.968 
[epoch:61, iter:4091] Loss: 0.103 | Acc: 0.968 
[epoch:61, iter:4092] Loss: 0.107 | Acc: 0.968 
[epoch:61, iter:4093] Loss: 0.104 | Acc: 0.968 
[epoch:61, iter:4094] Loss: 0.107 | Acc: 0.967 
[epoch:61, iter:4095] Loss: 0.111 | Acc: 0.966 
[epoch:61, iter:4096] Loss: 0.111 | Acc: 0.965 
[epoch:61, iter:4097] Loss: 0.110 | Acc: 0.965 
[epoch:61, iter:4098] Loss: 0.109 | Acc: 0.965 
[epoch:61, iter:4099] Loss: 0.108 | Acc: 0.966 
[epoch:61, iter:4100] Loss: 0.108 | Acc: 0.966 
[epoch:61, iter:4101] Loss: 0.106 | Acc: 0.967 
[epoch:61, iter:4102] Loss: 0.106 | Acc: 0.967 
[epoch:61, iter:4103] Loss: 0.106 | Acc: 0.967 
[epoch:61, iter:4104] Loss: 0.108 | Acc: 0.966 
[epoch:61, iter:4105] Loss: 0.107 | Acc: 0.967 
[epoch:61, iter:4106] Loss: 0.107 | Acc:

[epoch:63, iter:4256] Loss: 0.099 | Acc: 0.969 
[epoch:63, iter:4257] Loss: 0.098 | Acc: 0.969 
[epoch:63, iter:4258] Loss: 0.098 | Acc: 0.969 
[epoch:63, iter:4259] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4260] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4261] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4262] Loss: 0.098 | Acc: 0.970 
[epoch:63, iter:4263] Loss: 0.098 | Acc: 0.970 
[epoch:63, iter:4264] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4265] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4266] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4267] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4268] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4269] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4270] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4271] Loss: 0.098 | Acc: 0.969 
[epoch:63, iter:4272] Loss: 0.097 | Acc: 0.970 
[epoch:63, iter:4273] Loss: 0.098 | Acc: 0.969 
[epoch:63, iter:4274] Loss: 0.098 | Acc: 0.970 
[epoch:63, iter:4275] Loss: 0.098 | Acc: 0.969 
[epoch:63, iter:4276] Loss: 0.099 | Acc:

[epoch:66, iter:4424] Loss: 0.109 | Acc: 0.967 
[epoch:66, iter:4425] Loss: 0.106 | Acc: 0.970 
[epoch:66, iter:4426] Loss: 0.104 | Acc: 0.971 
[epoch:66, iter:4427] Loss: 0.111 | Acc: 0.968 
[epoch:66, iter:4428] Loss: 0.108 | Acc: 0.968 
[epoch:66, iter:4429] Loss: 0.111 | Acc: 0.967 
[epoch:66, iter:4430] Loss: 0.107 | Acc: 0.968 
[epoch:66, iter:4431] Loss: 0.106 | Acc: 0.969 
[epoch:66, iter:4432] Loss: 0.106 | Acc: 0.968 
[epoch:66, iter:4433] Loss: 0.105 | Acc: 0.968 
[epoch:66, iter:4434] Loss: 0.106 | Acc: 0.968 
[epoch:66, iter:4435] Loss: 0.107 | Acc: 0.967 
[epoch:66, iter:4436] Loss: 0.108 | Acc: 0.967 
[epoch:66, iter:4437] Loss: 0.106 | Acc: 0.966 
[epoch:66, iter:4438] Loss: 0.105 | Acc: 0.967 
[epoch:66, iter:4439] Loss: 0.103 | Acc: 0.968 
[epoch:66, iter:4440] Loss: 0.103 | Acc: 0.968 
[epoch:66, iter:4441] Loss: 0.101 | Acc: 0.969 
[epoch:66, iter:4442] Loss: 0.099 | Acc: 0.970 
[epoch:66, iter:4443] Loss: 0.099 | Acc: 0.971 
[epoch:66, iter:4444] Loss: 0.097 | Acc:

[epoch:68, iter:4593] Loss: 0.098 | Acc: 0.969 
[epoch:68, iter:4594] Loss: 0.099 | Acc: 0.969 
[epoch:68, iter:4595] Loss: 0.099 | Acc: 0.969 
[epoch:68, iter:4596] Loss: 0.099 | Acc: 0.969 
[epoch:68, iter:4597] Loss: 0.098 | Acc: 0.969 
[epoch:68, iter:4598] Loss: 0.098 | Acc: 0.969 
[epoch:68, iter:4599] Loss: 0.099 | Acc: 0.969 
[epoch:68, iter:4600] Loss: 0.099 | Acc: 0.969 
[epoch:68, iter:4601] Loss: 0.099 | Acc: 0.969 
[epoch:68, iter:4602] Loss: 0.100 | Acc: 0.969 
[epoch:68, iter:4603] Loss: 0.100 | Acc: 0.969 
[epoch:68, iter:4604] Loss: 0.099 | Acc: 0.969 
[epoch:68, iter:4605] Loss: 0.098 | Acc: 0.969 
[epoch:68, iter:4606] Loss: 0.098 | Acc: 0.969 
[epoch:68, iter:4607] Loss: 0.098 | Acc: 0.970 
[epoch:68, iter:4608] Loss: 0.098 | Acc: 0.970 
[epoch:68, iter:4609] Loss: 0.098 | Acc: 0.970 
[epoch:68, iter:4610] Loss: 0.098 | Acc: 0.970 
[epoch:68, iter:4611] Loss: 0.098 | Acc: 0.970 
[epoch:68, iter:4612] Loss: 0.098 | Acc: 0.970 
[epoch:68, iter:4613] Loss: 0.097 | Acc:

val acc：0.933


In [21]:
net5_new = train_sch(5)
val5_pred = pred_val(5)

Start Training, resnet!

Epoch: 1
[epoch:1, iter:1] Loss: 3.253 | Acc: 0.105 
[epoch:1, iter:2] Loss: 4.688 | Acc: 0.221 
[epoch:1, iter:3] Loss: 6.265 | Acc: 0.212 
[epoch:1, iter:4] Loss: 5.863 | Acc: 0.174 
[epoch:1, iter:5] Loss: 5.457 | Acc: 0.153 
[epoch:1, iter:6] Loss: 5.298 | Acc: 0.137 
[epoch:1, iter:7] Loss: 5.006 | Acc: 0.129 
[epoch:1, iter:8] Loss: 4.742 | Acc: 0.149 
[epoch:1, iter:9] Loss: 4.513 | Acc: 0.168 
[epoch:1, iter:10] Loss: 4.347 | Acc: 0.183 
[epoch:1, iter:11] Loss: 4.178 | Acc: 0.198 
[epoch:1, iter:12] Loss: 4.041 | Acc: 0.211 
[epoch:1, iter:13] Loss: 3.918 | Acc: 0.221 
[epoch:1, iter:14] Loss: 3.813 | Acc: 0.229 
[epoch:1, iter:15] Loss: 3.722 | Acc: 0.227 
[epoch:1, iter:16] Loss: 3.644 | Acc: 0.225 
[epoch:1, iter:17] Loss: 3.580 | Acc: 0.220 
[epoch:1, iter:18] Loss: 3.513 | Acc: 0.227 
[epoch:1, iter:19] Loss: 3.446 | Acc: 0.235 
[epoch:1, iter:20] Loss: 3.392 | Acc: 0.242 
[epoch:1, iter:21] Loss: 3.343 | Acc: 0.247 
[epoch:1, iter:22] Loss: 3.300

[epoch:3, iter:179] Loss: 1.595 | Acc: 0.545 
[epoch:3, iter:180] Loss: 1.592 | Acc: 0.546 
[epoch:3, iter:181] Loss: 1.588 | Acc: 0.548 
[epoch:3, iter:182] Loss: 1.587 | Acc: 0.548 
[epoch:3, iter:183] Loss: 1.582 | Acc: 0.548 
[epoch:3, iter:184] Loss: 1.581 | Acc: 0.548 
[epoch:3, iter:185] Loss: 1.580 | Acc: 0.549 
[epoch:3, iter:186] Loss: 1.580 | Acc: 0.548 
[epoch:3, iter:187] Loss: 1.576 | Acc: 0.550 
[epoch:3, iter:188] Loss: 1.571 | Acc: 0.551 
[epoch:3, iter:189] Loss: 1.567 | Acc: 0.552 
[epoch:3, iter:190] Loss: 1.562 | Acc: 0.554 
[epoch:3, iter:191] Loss: 1.560 | Acc: 0.554 
[epoch:3, iter:192] Loss: 1.559 | Acc: 0.555 
[epoch:3, iter:193] Loss: 1.556 | Acc: 0.556 
[epoch:3, iter:194] Loss: 1.551 | Acc: 0.557 
[epoch:3, iter:195] Loss: 1.546 | Acc: 0.558 
[epoch:3, iter:196] Loss: 1.547 | Acc: 0.558 
[epoch:3, iter:197] Loss: 1.546 | Acc: 0.558 
[epoch:3, iter:198] Loss: 1.543 | Acc: 0.559 
[epoch:3, iter:199] Loss: 1.542 | Acc: 0.560 
[epoch:3, iter:200] Loss: 1.538 | 

[epoch:6, iter:354] Loss: 0.924 | Acc: 0.731 
[epoch:6, iter:355] Loss: 0.922 | Acc: 0.730 
[epoch:6, iter:356] Loss: 0.923 | Acc: 0.730 
[epoch:6, iter:357] Loss: 0.923 | Acc: 0.730 
[epoch:6, iter:358] Loss: 0.924 | Acc: 0.731 
[epoch:6, iter:359] Loss: 0.922 | Acc: 0.731 
[epoch:6, iter:360] Loss: 0.926 | Acc: 0.731 
[epoch:6, iter:361] Loss: 0.931 | Acc: 0.730 
[epoch:6, iter:362] Loss: 0.934 | Acc: 0.729 
[epoch:6, iter:363] Loss: 0.929 | Acc: 0.730 
[epoch:6, iter:364] Loss: 0.929 | Acc: 0.729 
[epoch:6, iter:365] Loss: 0.931 | Acc: 0.729 
[epoch:6, iter:366] Loss: 0.932 | Acc: 0.728 
[epoch:6, iter:367] Loss: 0.938 | Acc: 0.727 
[epoch:6, iter:368] Loss: 0.937 | Acc: 0.728 
[epoch:6, iter:369] Loss: 0.936 | Acc: 0.728 
[epoch:6, iter:370] Loss: 0.938 | Acc: 0.726 
[epoch:6, iter:371] Loss: 0.937 | Acc: 0.727 
[epoch:6, iter:372] Loss: 0.935 | Acc: 0.728 
[epoch:6, iter:373] Loss: 0.934 | Acc: 0.728 
[epoch:6, iter:374] Loss: 0.934 | Acc: 0.728 
[epoch:6, iter:375] Loss: 0.934 | 

[epoch:8, iter:531] Loss: 0.776 | Acc: 0.772 
[epoch:8, iter:532] Loss: 0.775 | Acc: 0.773 
[epoch:8, iter:533] Loss: 0.774 | Acc: 0.773 
[epoch:8, iter:534] Loss: 0.774 | Acc: 0.773 
[epoch:8, iter:535] Loss: 0.776 | Acc: 0.772 
[epoch:8, iter:536] Loss: 0.774 | Acc: 0.773 
[epoch:8, iter:537] Loss: 0.774 | Acc: 0.773 
[epoch:8, iter:538] Loss: 0.775 | Acc: 0.773 
[epoch:8, iter:539] Loss: 0.775 | Acc: 0.773 
[epoch:8, iter:540] Loss: 0.777 | Acc: 0.773 
[epoch:8, iter:541] Loss: 0.778 | Acc: 0.773 
[epoch:8, iter:542] Loss: 0.776 | Acc: 0.773 
[epoch:8, iter:543] Loss: 0.777 | Acc: 0.773 
[epoch:8, iter:544] Loss: 0.777 | Acc: 0.773 
Waiting Test!
val acc：0.555

Epoch: 9
[epoch:9, iter:545] Loss: 0.719 | Acc: 0.777 
[epoch:9, iter:546] Loss: 0.720 | Acc: 0.779 
[epoch:9, iter:547] Loss: 0.717 | Acc: 0.777 
[epoch:9, iter:548] Loss: 0.726 | Acc: 0.782 
[epoch:9, iter:549] Loss: 0.739 | Acc: 0.774 
[epoch:9, iter:550] Loss: 0.726 | Acc: 0.780 
[epoch:9, iter:551] Loss: 0.729 | Acc: 0.7

[epoch:11, iter:704] Loss: 0.639 | Acc: 0.806 
[epoch:11, iter:705] Loss: 0.644 | Acc: 0.805 
[epoch:11, iter:706] Loss: 0.639 | Acc: 0.806 
[epoch:11, iter:707] Loss: 0.641 | Acc: 0.805 
[epoch:11, iter:708] Loss: 0.639 | Acc: 0.805 
[epoch:11, iter:709] Loss: 0.636 | Acc: 0.807 
[epoch:11, iter:710] Loss: 0.638 | Acc: 0.807 
[epoch:11, iter:711] Loss: 0.638 | Acc: 0.807 
[epoch:11, iter:712] Loss: 0.639 | Acc: 0.808 
[epoch:11, iter:713] Loss: 0.637 | Acc: 0.808 
[epoch:11, iter:714] Loss: 0.633 | Acc: 0.809 
[epoch:11, iter:715] Loss: 0.632 | Acc: 0.810 
[epoch:11, iter:716] Loss: 0.634 | Acc: 0.809 
[epoch:11, iter:717] Loss: 0.639 | Acc: 0.807 
[epoch:11, iter:718] Loss: 0.636 | Acc: 0.808 
[epoch:11, iter:719] Loss: 0.636 | Acc: 0.808 
[epoch:11, iter:720] Loss: 0.637 | Acc: 0.808 
[epoch:11, iter:721] Loss: 0.639 | Acc: 0.808 
[epoch:11, iter:722] Loss: 0.638 | Acc: 0.808 
[epoch:11, iter:723] Loss: 0.636 | Acc: 0.808 
[epoch:11, iter:724] Loss: 0.636 | Acc: 0.809 
[epoch:11, it

...

[epoch:58, iter:3923] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3924] Loss: 0.124 | Acc: 0.962 
[epoch:58, iter:3925] Loss: 0.124 | Acc: 0.962 
[epoch:58, iter:3926] Loss: 0.124 | Acc: 0.962 
[epoch:58, iter:3927] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3928] Loss: 0.124 | Acc: 0.962 
[epoch:58, iter:3929] Loss: 0.124 | Acc: 0.962 
[epoch:58, iter:3930] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3931] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3932] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3933] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3934] Loss: 0.124 | Acc: 0.962 
[epoch:58, iter:3935] Loss: 0.125 | Acc: 0.962 
[epoch:58, iter:3936] Loss: 0.126 | Acc: 0.962 
[epoch:58, iter:3937] Loss: 0.126 | Acc: 0.962 
[epoch:58, iter:3938] Loss: 0.126 | Acc: 0.962 
[epoch:58, iter:3939] Loss: 0.126 | Acc: 0.962 
[epoch:58, iter:3940] Loss: 0.127 | Acc: 0.962 
[epoch:58, iter:3941] Loss: 0.127 | Acc: 0.962 
[epoch:58, iter:3942] Loss: 0.127 | Acc: 0.962 
[epoch:58, iter:3943] Loss: 0.126 | Acc:

[epoch:61, iter:4091] Loss: 0.112 | Acc: 0.966 
[epoch:61, iter:4092] Loss: 0.115 | Acc: 0.965 
[epoch:61, iter:4093] Loss: 0.115 | Acc: 0.965 
[epoch:61, iter:4094] Loss: 0.114 | Acc: 0.965 
[epoch:61, iter:4095] Loss: 0.115 | Acc: 0.964 
[epoch:61, iter:4096] Loss: 0.116 | Acc: 0.963 
[epoch:61, iter:4097] Loss: 0.113 | Acc: 0.965 
[epoch:61, iter:4098] Loss: 0.112 | Acc: 0.964 
[epoch:61, iter:4099] Loss: 0.114 | Acc: 0.964 
[epoch:61, iter:4100] Loss: 0.113 | Acc: 0.965 
[epoch:61, iter:4101] Loss: 0.111 | Acc: 0.965 
[epoch:61, iter:4102] Loss: 0.110 | Acc: 0.966 
[epoch:61, iter:4103] Loss: 0.109 | Acc: 0.966 
[epoch:61, iter:4104] Loss: 0.110 | Acc: 0.966 
[epoch:61, iter:4105] Loss: 0.109 | Acc: 0.966 
[epoch:61, iter:4106] Loss: 0.108 | Acc: 0.967 
[epoch:61, iter:4107] Loss: 0.108 | Acc: 0.967 
[epoch:61, iter:4108] Loss: 0.109 | Acc: 0.967 
[epoch:61, iter:4109] Loss: 0.109 | Acc: 0.967 
[epoch:61, iter:4110] Loss: 0.109 | Acc: 0.967 
[epoch:61, iter:4111] Loss: 0.107 | Acc:

[epoch:63, iter:4260] Loss: 0.106 | Acc: 0.967 
[epoch:63, iter:4261] Loss: 0.105 | Acc: 0.967 
[epoch:63, iter:4262] Loss: 0.105 | Acc: 0.968 
[epoch:63, iter:4263] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4264] Loss: 0.103 | Acc: 0.969 
[epoch:63, iter:4265] Loss: 0.103 | Acc: 0.969 
[epoch:63, iter:4266] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4267] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4268] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4269] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4270] Loss: 0.103 | Acc: 0.968 
[epoch:63, iter:4271] Loss: 0.103 | Acc: 0.968 
[epoch:63, iter:4272] Loss: 0.105 | Acc: 0.968 
[epoch:63, iter:4273] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4274] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4275] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4276] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4277] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4278] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4279] Loss: 0.104 | Acc: 0.968 
[epoch:63, iter:4280] Loss: 0.104 | Acc:

[epoch:66, iter:4428] Loss: 0.104 | Acc: 0.967 
[epoch:66, iter:4429] Loss: 0.100 | Acc: 0.970 
[epoch:66, iter:4430] Loss: 0.105 | Acc: 0.968 
[epoch:66, iter:4431] Loss: 0.107 | Acc: 0.967 
[epoch:66, iter:4432] Loss: 0.106 | Acc: 0.967 
[epoch:66, iter:4433] Loss: 0.105 | Acc: 0.968 
[epoch:66, iter:4434] Loss: 0.108 | Acc: 0.967 
[epoch:66, iter:4435] Loss: 0.108 | Acc: 0.968 
[epoch:66, iter:4436] Loss: 0.105 | Acc: 0.969 
[epoch:66, iter:4437] Loss: 0.105 | Acc: 0.969 
[epoch:66, iter:4438] Loss: 0.104 | Acc: 0.969 
[epoch:66, iter:4439] Loss: 0.103 | Acc: 0.970 
[epoch:66, iter:4440] Loss: 0.102 | Acc: 0.970 
[epoch:66, iter:4441] Loss: 0.101 | Acc: 0.969 
[epoch:66, iter:4442] Loss: 0.104 | Acc: 0.968 
[epoch:66, iter:4443] Loss: 0.103 | Acc: 0.969 
[epoch:66, iter:4444] Loss: 0.102 | Acc: 0.969 
[epoch:66, iter:4445] Loss: 0.103 | Acc: 0.969 
[epoch:66, iter:4446] Loss: 0.103 | Acc: 0.969 
[epoch:66, iter:4447] Loss: 0.103 | Acc: 0.968 
[epoch:66, iter:4448] Loss: 0.103 | Acc:

[epoch:68, iter:4598] Loss: 0.100 | Acc: 0.971 
[epoch:68, iter:4599] Loss: 0.100 | Acc: 0.971 
[epoch:68, iter:4600] Loss: 0.101 | Acc: 0.970 
[epoch:68, iter:4601] Loss: 0.101 | Acc: 0.970 
[epoch:68, iter:4602] Loss: 0.101 | Acc: 0.970 
[epoch:68, iter:4603] Loss: 0.102 | Acc: 0.970 
[epoch:68, iter:4604] Loss: 0.101 | Acc: 0.970 
[epoch:68, iter:4605] Loss: 0.102 | Acc: 0.970 
[epoch:68, iter:4606] Loss: 0.102 | Acc: 0.970 
[epoch:68, iter:4607] Loss: 0.101 | Acc: 0.970 
[epoch:68, iter:4608] Loss: 0.101 | Acc: 0.970 
[epoch:68, iter:4609] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4610] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4611] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4612] Loss: 0.103 | Acc: 0.969 
[epoch:68, iter:4613] Loss: 0.103 | Acc: 0.969 
[epoch:68, iter:4614] Loss: 0.103 | Acc: 0.969 
[epoch:68, iter:4615] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4616] Loss: 0.102 | Acc: 0.969 
[epoch:68, iter:4617] Loss: 0.101 | Acc: 0.969 
[epoch:68, iter:4618] Loss: 0.102 | Acc:

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = ResNet18().to(device)

criterion = nn.CrossEntropyLoss()  

pre_epoch_list = [0,40,60]# previous running epoches
EPOCH_list = [40,60,70]   #count epoches
LR_list = [0.1,0.01,0.001]        

def train_sch_new():
    
    net = ResNet18().to(device)
    trainset = MyDataset("train.txt", transform = transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

    valset = MyDataset("val.txt", transform = transform_test)
    valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

    best_acc = 0  #initial best test accuracy
    print("Start Training, resnet!")  

    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-3)
    scheduler = CosineAnnealingLR_with_Restart(optimizer, T_max=4, T_mult=1.5, model=net, out_dir="./", take_snapshot=True, eta_min=1e-6)
    
    with open("acc.txt", "w") as f:
        with open("log.txt", "w")as f2:
            for ep_num in range(3):
                pre_epoch = pre_epoch_list[ep_num]
                EPOCH = EPOCH_list[ep_num]
                LR = LR_list[ep_num]
                optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
                
                for epoch in range(pre_epoch, EPOCH):
                    scheduler.step()
                    print('\nEpoch: %d' % (epoch + 1))
                    net.train()
                    sum_loss = 0.0
                    correct = 0.0
                    total = 0.0
                    for i, data in enumerate(trainloader, 0):
                        length = len(trainloader)
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        optimizer.zero_grad()

                        # forward + backward
                        outputs = net(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                        # print loss and acc
                        sum_loss += loss.item()
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += predicted.eq(labels.data).cpu().sum()
                        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.03f '
                              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), float(correct) / float(total)))
                        f2.write('%03d  %05d |Loss: %.03f | Acc: %.03f '
                              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1),  float(correct / total)))
                        f2.write('\n')
                        f2.flush()

                    # val
                    print("Waiting Test!")
                    with torch.no_grad():
                        correct = 0
                        total = 0
                        for data in valloader:
                            net.eval()
                            images, labels = data
                            images, labels = images.to(device), labels.to(device)
                            outputs = net(images)
                            _, predicted = torch.max(outputs.data, 1)
                            total += labels.size(0)
                            correct += (predicted == labels).sum()
                        print('val acc：%.03f' % (int(correct) / int(total)))
                        acc = int(correct) / int(total)


                        if acc > best_acc:
                            torch.save(net.state_dict(), './net_total.pth')
                            best_acc = acc
                            print('Saving model......')

                        f.write("EPOCH=%03d,Accuracy= %.03f" % (epoch + 1, acc))
                        f.write('\n')
                        f.flush()

            print("Training Finished, Total EPOCH=%d" % EPOCH)
    return net

In [ ]:
net_total = train_sch_new()

Start Training, resnet!

Epoch: 1
[epoch:1, iter:1] Loss: 3.093 | Acc: 0.070 
[epoch:1, iter:2] Loss: 5.225 | Acc: 0.234 
[epoch:1, iter:3] Loss: 5.245 | Acc: 0.227 
[epoch:1, iter:4] Loss: 4.982 | Acc: 0.189 
[epoch:1, iter:5] Loss: 4.722 | Acc: 0.165 
[epoch:1, iter:6] Loss: 4.465 | Acc: 0.150 
[epoch:1, iter:7] Loss: 4.237 | Acc: 0.172 
[epoch:1, iter:8] Loss: 4.055 | Acc: 0.188 
[epoch:1, iter:9] Loss: 3.884 | Acc: 0.211 
[epoch:1, iter:10] Loss: 3.757 | Acc: 0.223 
[epoch:1, iter:11] Loss: 3.628 | Acc: 0.240 
[epoch:1, iter:12] Loss: 3.525 | Acc: 0.247 
[epoch:1, iter:13] Loss: 3.433 | Acc: 0.256 
[epoch:1, iter:14] Loss: 3.368 | Acc: 0.251 
[epoch:1, iter:15] Loss: 3.300 | Acc: 0.252 
[epoch:1, iter:16] Loss: 3.250 | Acc: 0.250 
[epoch:1, iter:17] Loss: 3.214 | Acc: 0.253 
[epoch:1, iter:18] Loss: 3.182 | Acc: 0.255 
[epoch:1, iter:19] Loss: 3.137 | Acc: 0.261 
[epoch:1, iter:20] Loss: 3.105 | Acc: 0.263 
[epoch:1, iter:21] Loss: 3.068 | Acc: 0.267 
[epoch:1, iter:22] Loss: 3.038

[epoch:3, iter:179] Loss: 1.430 | Acc: 0.593 
[epoch:3, iter:180] Loss: 1.428 | Acc: 0.593 
[epoch:3, iter:181] Loss: 1.423 | Acc: 0.595 
[epoch:3, iter:182] Loss: 1.435 | Acc: 0.592 
[epoch:3, iter:183] Loss: 1.440 | Acc: 0.591 
[epoch:3, iter:184] Loss: 1.434 | Acc: 0.593 
[epoch:3, iter:185] Loss: 1.429 | Acc: 0.594 
[epoch:3, iter:186] Loss: 1.427 | Acc: 0.593 
[epoch:3, iter:187] Loss: 1.428 | Acc: 0.592 
[epoch:3, iter:188] Loss: 1.421 | Acc: 0.595 
[epoch:3, iter:189] Loss: 1.421 | Acc: 0.594 
[epoch:3, iter:190] Loss: 1.422 | Acc: 0.593 
[epoch:3, iter:191] Loss: 1.419 | Acc: 0.593 
[epoch:3, iter:192] Loss: 1.417 | Acc: 0.594 
[epoch:3, iter:193] Loss: 1.414 | Acc: 0.595 
[epoch:3, iter:194] Loss: 1.411 | Acc: 0.597 
[epoch:3, iter:195] Loss: 1.408 | Acc: 0.597 
[epoch:3, iter:196] Loss: 1.408 | Acc: 0.596 
[epoch:3, iter:197] Loss: 1.403 | Acc: 0.598 
[epoch:3, iter:198] Loss: 1.397 | Acc: 0.600 
[epoch:3, iter:199] Loss: 1.391 | Acc: 0.601 
[epoch:3, iter:200] Loss: 1.390 | 

[epoch:5, iter:355] Loss: 0.953 | Acc: 0.721 
[epoch:5, iter:356] Loss: 0.940 | Acc: 0.725 
[epoch:5, iter:357] Loss: 0.942 | Acc: 0.725 
[epoch:5, iter:358] Loss: 0.939 | Acc: 0.725 
[epoch:5, iter:359] Loss: 0.930 | Acc: 0.728 
[epoch:5, iter:360] Loss: 0.924 | Acc: 0.731 
[epoch:5, iter:361] Loss: 0.924 | Acc: 0.731 
[epoch:5, iter:362] Loss: 0.929 | Acc: 0.729 
[epoch:5, iter:363] Loss: 0.927 | Acc: 0.730 
[epoch:5, iter:364] Loss: 0.923 | Acc: 0.729 
[epoch:5, iter:365] Loss: 0.921 | Acc: 0.729 
[epoch:5, iter:366] Loss: 0.920 | Acc: 0.729 
[epoch:5, iter:367] Loss: 0.919 | Acc: 0.730 
[epoch:5, iter:368] Loss: 0.919 | Acc: 0.730 
[epoch:5, iter:369] Loss: 0.917 | Acc: 0.730 
[epoch:5, iter:370] Loss: 0.918 | Acc: 0.731 
[epoch:5, iter:371] Loss: 0.920 | Acc: 0.730 
[epoch:5, iter:372] Loss: 0.916 | Acc: 0.731 
[epoch:5, iter:373] Loss: 0.914 | Acc: 0.732 
[epoch:5, iter:374] Loss: 0.913 | Acc: 0.732 
[epoch:5, iter:375] Loss: 0.913 | Acc: 0.732 
[epoch:5, iter:376] Loss: 0.911 | 

[epoch:7, iter:532] Loss: 0.724 | Acc: 0.790 
[epoch:7, iter:533] Loss: 0.721 | Acc: 0.791 
[epoch:7, iter:534] Loss: 0.724 | Acc: 0.790 
[epoch:7, iter:535] Loss: 0.721 | Acc: 0.790 
[epoch:7, iter:536] Loss: 0.720 | Acc: 0.790 
[epoch:7, iter:537] Loss: 0.719 | Acc: 0.791 
[epoch:7, iter:538] Loss: 0.722 | Acc: 0.790 
[epoch:7, iter:539] Loss: 0.724 | Acc: 0.790 
[epoch:7, iter:540] Loss: 0.728 | Acc: 0.789 
[epoch:7, iter:541] Loss: 0.726 | Acc: 0.790 
[epoch:7, iter:542] Loss: 0.729 | Acc: 0.789 
[epoch:7, iter:543] Loss: 0.728 | Acc: 0.790 
[epoch:7, iter:544] Loss: 0.732 | Acc: 0.788 
[epoch:7, iter:545] Loss: 0.730 | Acc: 0.788 
[epoch:7, iter:546] Loss: 0.730 | Acc: 0.788 
[epoch:7, iter:547] Loss: 0.731 | Acc: 0.787 
[epoch:7, iter:548] Loss: 0.730 | Acc: 0.787 
[epoch:7, iter:549] Loss: 0.729 | Acc: 0.787 
[epoch:7, iter:550] Loss: 0.732 | Acc: 0.787 
[epoch:7, iter:551] Loss: 0.731 | Acc: 0.787 
[epoch:7, iter:552] Loss: 0.730 | Acc: 0.788 
[epoch:7, iter:553] Loss: 0.729 | 

In [40]:
net_total = ResNet18().eval().to(device)
net_total.load_state_dict(torch.load('model/net_total.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [13]:
testloader

In [12]:
# testset prediction

total_test_predict = torch.empty(0).to(device)
    
with torch.no_grad(): 
    for data in testloader:
        images = data
        images = images.to(device)
        outputs = net_total(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test_predict = torch.cat([total_test_predict, outputs])
test_pred = pd.DataFrame(total_test_predict.tolist())
# val.to_csv('./val_pred_%d.csv' % (k_fold))

BrokenPipeError: [Errno 32] Broken pipe

In [29]:
# testset prediction

total_test_predict = torch.empty(0).to(device)
    
with torch.no_grad():
    for data in testloader:
        images = data
        images = images.to(device)
        outputs = net_total(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test_predict = torch.cat([total_test_predict, outputs])
test_pred = pd.DataFrame(total_test_predict.tolist())
# val.to_csv('./val_pred_%d.csv' % (k_fold))

In [42]:
# stacking the 5-fold valset prediction

train_pred = pd.concat([val1_pred,val2_pred,val3_pred,val4_pred,val5_pred])
train_pred.to_csv("image2_train_pred.csv", index=False)
test_pred.to_csv("image2_test_pred.csv", index=False)